In [ ]:
## Ambiente beakerx
# !pip install py2neo
import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import torch, sqlite3, asyncio
import os, re, time, traceback, json
import warnings, csv, sys, pip, string

from PIL import Image
from io import BytesIO
from pprint import pprint
from string import Formatter
from bs4 import BeautifulSoup
from datetime import datetime
from datetime import timedelta
from collections import Counter
from collections import defaultdict
from flask import render_template_string
from neo4j import GraphDatabase
from py2neo import Graph, Node, Relationship
from IPython.display import clear_output, display, HTML

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException, TimeoutException

## Configurar exibição dos dataframes do pandas na tela
pd.set_option('display.max_colwidth', None)
pd.set_option('colheader_justify', 'left')
pd.set_option('display.max_rows', 600)

delay = 10

In [ ]:
# Define database credentials and URI
uri = "bolt://localhost:7687"
user = "neo4j"
password = "password"


def connect_driver():
    '''
    Função 1: Conecta ao servidor do CNPq para busca de currículo
    '''
    print(f'Conectando com o servidor do CNPq...')
    # print(f'Iniciada extração de {len(lista_nomes)} currículos')
    
    ## https://www.selenium.dev/documentation/pt-br/webdriver/browser_manipulation/
    options   = Options()
    # options.add_argument("--headless")
    driver   = webdriver.Chrome(options=options)
    url_buscaespecialista = 'http://buscatextual.cnpq.br/buscatextual/busca.do?buscarDoutores=true&buscarDemais=true&textoBusca='
    driver.get(url_buscaespecialista) # acessa a url de busca do CNPQ   
    
    driver.set_window_position(-20, -10)
    driver.set_window_size(170, 1896)
    driver.mouse = webdriver.ActionChains(driver)
    # url        = browser.command_executor._url #"http://127.0.0.1:60622/hub"
    # session_id = browser.session_id            #'4e167f26-dc1d-4f51-a207-f761eaf73c31'

    # return browser, url, session_id
    return driver


def persist_to_neo4j(header_data):
    try:
        graph = Graph("bolt://localhost:7687", auth=("neo4j", "password"))
    except:
        print('Erro ao conectar ao Neo4j')
    header_node = Node("Curriculum", title=header_data['title'].split('(')[1].strip(')'), meta_keywords=header_data['meta_keywords'])
    graph.create(header_node)
    print(type(header_node))

    return header_node

In [ ]:
def fill_name(driver, delay, NOME):
    '''
    Função 2: passa o nome para campo de busca
    '''
    try:
        nome = lambda: driver.find_element(By.CSS_SELECTOR, ("#textoBusca"))
        nome().send_keys(Keys.CONTROL + "a")
        nome().send_keys(NOME)
        # driver.find_element(By.CSS_SELECTOR, ("#textoBusca")).send_keys(Keys.ENTER)
        seletorcss='div.layout-cell-12:nth-child(8) > div:nth-child(1) > div:nth-child(1) > a:nth-child(1)'
        # seletorcss="#textoBusca"  
        
        time.sleep(0.5)
        WebDriverWait(driver, delay).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, seletorcss))).click()
        seletorcss="#botaoBuscaFiltros"  
        
        WebDriverWait(driver, delay).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, seletorcss)))
        
    except Exception as e:
        traceback_str = ''.join(traceback.format_tb(e.__traceback__))          
        print('Erro ao preencher nome no campo de busca, pausando por 1 segundo')
        print(e,traceback_str)
        # print(e)
        # time.sleep(1.5)

In [ ]:
def retry(func,
          expected_ex_type=Exception,
          limit=0,
          wait_ms=100,
          wait_increase_ratio=2,
          logger=None,
          on_exhaust="throw"
          ):
    """
    Retry a function invocation until no exception occurs
    :param func: function to invoke
    :param expected_ex_type: retry only if exception is subclass of this type
    :param limit: maximum number of invocation attempts, 0 for unlimited attempts.
    :param wait_ms: initial wait time after each attempt in milliseconds.
    :param wait_increase_ratio: increase wait period by multiplying this value after each attempt.
    :param logger: if not None, retry attempts will be logged to this logging.logger
    :param on_exhaust: return value when retry attempts exhausted. Default is "throw" which will rethrow the exception
                 of the last attempt.
    :return: result of first successful invocation
    :raises: last invocation exception if attempts exhausted or exception is not an instance of ex_type
    """

    attempt = 1
    while True:
        try:
            return func()
        except Exception as ex:
            if not isinstance(ex, expected_ex_type):
                raise ex

            if logger:
                logger.error("Failed execution attempt #%d", attempt, exc_info=ex)

            # check if attempts exhausted
            if 0 < limit <= attempt:
                if logger:
                    logger.warning("Attempt limit (%d) reached", limit)
                if on_exhaust == "throw":
                    raise ex
                return on_exhaust

            # prepare for next attempt
            attempt += 1
            if logger:
                logger.info("Waiting %d ms before attempt #%d", wait_ms, attempt)
            time.sleep(wait_ms / 1000)
            wait_ms *= wait_increase_ratio

def paginar(browser):
    try:
        css_paginacao="div.paginacao:nth-child(2)"  
        WebDriverWait(browser, delay).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, css_paginacao)))
        paginacao = browser.find_element(By.CSS_SELECTOR, css_paginacao)
        paginas=paginacao.text.split(' ')
        remover=['','anterior','...']
        numpaginas = [x for x in paginas if x not in remover]
        # print('NumPáginas interno:',numpaginas)
    except Exception as e:
        print('Erro ao utilizar função paginar():', e)
    return numpaginas

In [ ]:
def achar_busca(driver, delay):
    '''
    Função 3: clica no botão Buscar Currículo
    '''
    delay=10
    try:
        limite=5
        xpath_nome = "/html/body/form/div/div[4]/div/div/div/div[3]/div/div[3]/ol/li/b"
        retry(WebDriverWait(driver, delay).until(
            EC.element_to_be_clickable((By.XPATH, xpath_nome))),
            #    expected_ex_type=ZeroDivisionError, 
               wait_ms=20,
               limit=limite, 
            #    logger=logger, 
               on_exhaust=(f'Problema ao acessar ao servidor do CNpQ função definir_filtros(). {limite} tentativas sem sucesso.'))   

        link_nome  = driver.find_element(By.XPATH, xpath_nome)
        # ActionChains(browser).move_to_element(link_nome).perform()
        
        # Avaliar 'Stale file handle'
        if link_nome.text == None:
            xpath_nome = '/html/body/form/div/div[4]/div/div/div/div[3]/div/div[3]/ol/li'
            
            print('Sem resposta do servidor, tentando novamente...')
            retry(WebDriverWait(driver, delay).until(
                EC.element_to_be_clickable((By.XPATH, xpath_nome))),
               #expected_ex_type=ZeroDivisionError, 
               wait_ms=200,
               limit=limite, 
               #logger=logger, 
               on_exhaust=(f'Problema ao acessar ao servidor do CNpQ função definir_filtros(). {limite} tentativas sem sucesso.'))   

        return link_nome

    except TimeoutException as t:
        print(f'Erro de conexão durante achar_busca(). Reiniciando a função dentro de 2 segundos...')
        time.sleep(2)

In [ ]:
def open_curriculum(driver,elm_vinculo):
    link_nome     = achar_busca(driver, delay)
    window_before = driver.current_window_handle

    if str(elm_vinculo) == 'nan':
        print('Vínculo não encontrado, passando ao próximo nome...')
        falhas.append(nome_falha)
        duvidas.append(duvida)
        tipo_erro.append(erro)
        # print(nome_falha)
        # print(erro)
        # clear_output(wait=True)
        raise Exception
    print('Vínculo encontrado no currículo de nome:',elm_vinculo.text)

    ## Clicar no botão abrir currículo e mudar de aba
    try:
        ## Aguarda, encontra, clica em buscar nome
        link_nome    = achar_busca(driver, delay)
        nome_buscado = []
        nome_achado  = []
        nome_buscado.append(NOME)
    except Exception as e:
        print('Erro')
        print(e)
        
    if link_nome.text == None:
        xpath_nome = '/html/body/form/div/div[4]/div/div/div/div[3]/div/div[3]/ol/li'
        # 'Stale file handle'
        print('Ainda sem resposta do servidor, tentando novamente...')
        retry(WebDriverWait(driver, delay).until(
            EC.element_to_be_clickable((By.XPATH, xpath_nome))),
        #    expected_ex_type=ZeroDivisionError, 
        wait_ms=200,
        limit=limite, 
        #    logger=logger, 
        on_exhaust=(f'Problema ao acessar ao servidor do CNpQ função definir_filtros(). {limite} tentativas sem sucesso.'))
    try:
        ActionChains(driver).click(link_nome).perform()
        nome_achado.append(link_nome.text)
    except:
        print(f'Currículo não encontrado para: {NOME}.')

    retry(WebDriverWait(driver, delay).until(
        EC.element_to_be_clickable((By.CSS_SELECTOR, "#idbtnabrircurriculo"))),
        #    expected_ex_type=ZeroDivisionError, 
        wait_ms=200,
        limit=limite, 
        #    logger=logger, 
        on_exhaust=(f'Problema ao acessar ao servidor do CNpQ função definir_filtros(). {limite} tentativas sem sucesso.'))   

    # Clicar botão para abrir o currículo
    btn_abrir_curriculo = WebDriverWait(driver, delay).until(
        EC.element_to_be_clickable((By.CSS_SELECTOR, "#idbtnabrircurriculo")))
    time.sleep(0.2)

    ActionChains(driver).click(btn_abrir_curriculo).perform()

    ## Gerenciamento das janelas abertas no browser
    WebDriverWait(driver, delay).until(EC.number_of_windows_to_be(2))
    window_after = driver.window_handles
    new_window   = [x for x in window_after if x != window_before][0]
    driver.switch_to.window(new_window)

    # Pega o código fonte da página
    page_source = driver.page_source
    driver.quit()

    # Usa BeautifulSoup para analisar
    soup = BeautifulSoup(page_source, 'html.parser') 
    
    return soup

In [ ]:
def find_terms(NOME, instituicao, unidade, termo, driver, delay, limite):
    from selenium.common import exceptions
    from pyjarowinkler.distance import get_jaro_distance
    ignored_exceptions=(NoSuchElementException,StaleElementReferenceException,)
    
    ## Receber a quantidade de opções ao ler elementos de resultados
    duvidas   = []
    force_break_loop = False
    try:
        css_resultados = ".resultado"
        WebDriverWait(driver, delay).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, css_resultados)))
        resultados = driver.find_elements(By.CSS_SELECTOR, css_resultados)       
        ## Ler quantidade de resultados apresentados pela busca de nome
        try:
            css_qteresultados = ".tit_form > b:nth-child(1)"
            WebDriverWait(driver, delay).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, css_qteresultados)))                       
            soup = BeautifulSoup(driver.page_source, 'html.parser')
            div_element = soup.find('div', {'class': 'tit_form'})
            match = re.search(r'<b>(\d+)</b>', str(div_element))
            if match:
                qte_resultados = int(match.group(1))
                print(f'{qte_resultados} resultados para {NOME}')
            else:
                raise Exception
        except Exception as e:
            print('Erro ao ler a quantidade de resultados:')
            print(e)
            return np.NaN, NOME, np.NaN, e, driver
        
        ## Escolher função a partir da quantidade de resultados da lista apresentada na busca
        ## Ao achar clica no elemento elm_vinculo com link do nome para abrir o currículo
        numpaginas = paginar(driver)
        if numpaginas == [] and qte_resultados==1:
            # capturar link para o primeiro nome resultado da busca
            ## TO-DO inserir a crítica para o 'Stale file handle'
            try:
                css_linknome = ".resultado > ol:nth-child(1) > li:nth-child(1) > b:nth-child(1) > a:nth-child(1)"
                WebDriverWait(driver, delay).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, css_linknome)))            
                elm_vinculo  = driver.find_element(By.CSS_SELECTOR, css_linknome)
                nome_vinculo = elm_vinculo.text
            except Exception as e:
                print('Erro ao encontrar o primeiro resultado da lista de nomes:',e)
                return np.NaN, NOME, np.NaN, e, driver
            # print('Clicar no nome único:', nome_vinculo)
            try:
                retry(ActionChains(driver).click(elm_vinculo).perform(),
                       wait_ms=20,
                       limit=limite,
                       on_exhaust=(f'Problema ao clicar no link do nome. {limite} tentativas sem sucesso.'))   
            except Exception as e:
                print('Erro ao clicar no único nome encontrado anteriormente',e)
                return np.NaN, NOME, np.NaN, e, driver
        
        ## Quantidade de resultados até 10 currículos, acessados sem paginação
        else:
            print(f'{qte_resultados} currículos de homônimos em potencial...')
            numpaginas = paginar(driver)
            numpaginas.append('próximo')
            iteracoes=0
            ## iterar em cada página de resultados
            pagin = qte_resultados//10+1
            for i in range(pagin+1):
                # print(i,'/',pagin)
                iteracoes+=1
                try:
                    numpaginas = paginar(driver)
                    print(f'Iteração: {iteracoes}. Páginas sendo lidas: {numpaginas}')
                    css_resultados = ".resultado"
                    WebDriverWait(driver, delay).until(
                        EC.presence_of_element_located((By.CSS_SELECTOR, css_resultados)))
                    resultados = driver.find_elements(By.CSS_SELECTOR, css_resultados)
                except Exception as e:
                    print('Erro ao paginar:',e)
                ## iterar em cada resultado
                for n,i in enumerate(resultados):
                    linhas = i.text.split('\n\n')
                    # print(linhas)
                    if 'Stale file handle' in str(linhas):
                        return np.NaN, NOME, np.NaN, 'Stale file handle', driver
                    for m,linha in enumerate(linhas):
                        # print(f'\nOrdem da linha: {m+1}, de total de linhas {len(linhas)}')
                        # print('Conteúdo da linha:',linha.lower())
                        print(linha)
                        try:
                            if instituicao.lower() in linha.lower() or unidade.lower() in linha.lower() or termo.lower() in linha.lower():
                                # print('Vínculo encontrado!')
                                count=m
                                # print(' NOME:', NOME, type(NOME))
                                # test = linhas[count].split('\n')[0]
                                # print('TESTE:',test, type(test))
                                while get_jaro_distance(linhas[count].split('\n')[0], str(NOME)) < 0.75:
                                    count-=1
                                print('Identificado vínculo com o interesse de análise no resultado:', m+1)
                                nome_vinculo = linhas[count].strip()
                                print(f'    Achado: {nome_vinculo}')
                                try:
                                    css_vinculo = f".resultado > ol:nth-child(1) > li:nth-child({m+1}) > b:nth-child(1) > a:nth-child(1)"
                                    # print('\nCSS_SELECTOR usado:', css_vinculo)
                                    css_alvo = '.resultado > ol:nth-child(1) > li:nth-child(7) > b:nth-child(1) > a:nth-child(1)'
                                    WebDriverWait(driver, delay).until(EC.presence_of_element_located((By.CSS_SELECTOR, css_vinculo)))            
                                    elm_vinculo  = driver.find_element(By.CSS_SELECTOR, css_vinculo)
                                    nome_vinculo = elm_vinculo.text
                                    # print('Elemento retornado:',nome_vinculo)
                                    retry(ActionChains(driver).click(elm_vinculo).perform(),
                                        wait_ms=100,
                                        limit=limite,
                                        on_exhaust=(f'Problema ao clicar no link do nome. {limite} tentativas sem sucesso.'))            
                                except Exception as e:
                                    print('Erro ao achar o link do nome com múltiplos resultados')
                                    return np.NaN, NOME, np.NaN, e, driver
                                force_break_loop = True
                                break
                        except Exception as e2:
                            traceback_str = ''.join(traceback.format_tb(e2.__traceback__))
                            print('Erro ao procurar vínculo com currículos achados')    
                            print(e2,traceback_str)
                        ## Caso percorra todos elementos da lista e não encontre vínculo adiciona à dúvidas quanto ao nome
                        if m==(qte_resultados):
                            print(f'Não encontrada nenhuma referência à {instituicao} ou ao {unidade} ou ao termo {termo}')
                            duvidas.append(NOME)
                            # clear_output(wait=True)
                            # driver.quit()
                            continue
                    if force_break_loop:
                        break
                try:
                    prox = driver.find_element(By.PARTIAL_LINK_TEXT, 'próximo')
                    prox.click()
                except:
                    continue
        try:
            elm_vinculo.text
            # print(f'Nomes: {NOME} | {elm_vinculo.text}')
        except:
            return np.NaN, NOME, np.NaN, 'Vínculo não encontrado', driver
    except Exception as err:
        print('Erro ao sair da função procurar_vinculos()')
        print('Conteúdo do erro:',err)
        return np.NaN, NOME, np.NaN, err, driver
    
    return elm_vinculo, np.NaN, np.NaN, np.NaN, driver

In [ ]:
def listar_idlattes(df_secoes):
    df_idlattes = df_secoes[df_secoes['ROTULOS']=='ID Lattes:']
    return df_idlattes

def listar_artigos(df_secoes):
    df_idlattes = df_secoes[df_secoes['ROTULOS']=='Artigos completos publicados em periódicos']
    return df_idlattes

def contar_artigos(df_secoes):
    def count_year_occurrences(content):
        """Count the number of occurrences of four-digit years followed by a period in the given string.""" 
        if not isinstance(content, (str, bytes)):
            content = ' '.join(map(str, content))
        pattern = r'\b\d{4}\.'
        return len(re.findall(pattern, content))

    def extract_citations(content):
        if not isinstance(content, (str, bytes)):
            content = ' '.join(map(str, content))
        citation_pattern = r'Citações:([\d\|]+),'
        all_citations = re.findall(citation_pattern, content)
        total_citations = 0
        for citation_group in all_citations:
            numbers = map(int, citation_group.split('|'))
            total_citations += sum(numbers)
        return len(all_citations), total_citations

    df_secoes_contadas = df_secoes.copy()
    df_secoes_contadas['QTE_ARTIGOS'] = df_secoes['CONTEUDOS'].apply(count_year_occurrences)
    df_secoes_contadas['ARTIGOS_CITADOS'], df_secoes['SOMA_CITACOES'] = zip(*df_secoes['CONTEUDOS'].apply(extract_citations))

    return df_secoes_contadas

In [ ]:
## VERSÃO 01
import logging

def extrair_indices(layout_celula):
    indice = layout_celula.select_one('div.layout-cell-pad-5.text-align-right')
    valor = layout_celula.select_one('div.layout-cell.layout-cell-9 div.layout-cell-pad-5')
    
    if indice and valor:
        valores_extraidos = valor.text.split('<br class="clear">') if '<br class="clear">' in valor.text else valor.text.split('\n\t\t\n\t')
        return indice.text, valores_extraidos

def encontrar_subchave(title_wrapper):
    tags_relevantes  = ['ul', 'a', 'b']
    tags_encontradas = [(tag, title_wrapper.find(tag)) for tag in tags_relevantes]
    tags_ordenadas   = sorted(tags_encontradas, key=lambda x: x[1].sourceline if x[1] else float('inf'))

    return tags_ordenadas[0][1] if tags_ordenadas else None

def extrair_secao(title_wrapper):
    # Inicializa a lista de tags relevantes e a lista que armazenará os elementos encontrados
    tags_relevantes = ['h1', 'ul', 'b']
    tags_encontradas = []

    # Loop para encontrar os demais elementos relevantes
    for tag in tags_relevantes:
        elemento_encontrado = title_wrapper.select_one(tag)
        if elemento_encontrado:
            tags_encontradas.append((tag, elemento_encontrado))

    # Ordena os elementos encontrados com base na linha de origem no código fonte HTML
    tags_ordenadas = sorted(tags_encontradas, key=lambda x: x[1].sourceline if x[1] else float('inf'))

    # Retorna o primeiro elemento da lista ordenada, se houver elementos
    return tags_ordenadas[0][1] if tags_ordenadas else None

def extrair_titulo(title_wrapper):
    inst_back_texts = {}
    inst_back_elements = title_wrapper.find_all('div', class_='inst_back')
    for index, inst_back in enumerate(inst_back_elements):
        b_tag = inst_back.find('b')
        if b_tag:
            inst_back_texts[index] = b_tag.text.strip()

    return inst_back_texts if inst_back_texts else None

def extrair_dados(soup, verbose=False):
    nome_no = soup.select_one('div.infpessoa h2.nome').text if soup.select_one('div.infpessoa h2.nome') else None
    
    if not nome_no:
        logging.error("Nome do nó não encontrado. Abortando.")
        return
    
    dados_json = {nome_no: {}}
    celula_principal = soup.select_one('div.layout-cell-pad-main')

    title_wrappers = celula_principal.select('div.title-wrapper')
    if verbose:
        logging.info(f'{len(title_wrappers)} seções de dados lidas com sucesso.')
    
    for title_wrapper in title_wrappers:
        nome_secao = extrair_secao(title_wrapper)
        
        if nome_secao is None:
            continue
        
        if verbose:
            logging.info(f'Marcador de Seção: "{nome_secao.text}"')

        titulo = extrair_titulo(title_wrapper)
        if verbose:
            logging.info(f'Marcador de Título: "{titulo}"')
                    
        chave = nome_secao.text
        dados_json[nome_no][chave] = {}

        # A seleção agora ocorre dentro do contexto de title_wrapper, e não de celula_principal.
        celulas_layout = title_wrapper.select('div.layout-cell.layout-cell-12.data-cell')
        
        for celula_layout in celulas_layout:
            if celula_layout.find_all('div'):
                indice, valores = extrair_indices(celula_layout)
                
                if indice and valores:
                    dados_json[nome_no][chave][indice] = valores

    if verbose:
        logging.info(f"Total de índices extraídos: {len(dados_json[nome_no].keys())}")
        # Outros blocos de código para depuração e verbosidade
    return dados_json

In [ ]:
## VERSÃO 3 com erros ainda
import json
import logging
from bs4 import BeautifulSoup
from neo4j import GraphDatabase

# Initialize Logging
logging.basicConfig(level=logging.INFO)

def extrair_secao(title_wrapper):
    tags_relevantes = ['h1', 'ul', 'b']
    tags_encontradas = []
    for tag in tags_relevantes:
        elemento_encontrado = title_wrapper.select_one(tag)
        if elemento_encontrado:
            tags_encontradas.append((tag, elemento_encontrado))
    tags_ordenadas = sorted(tags_encontradas, key=lambda x: x[1].sourceline if x[1] else float('inf'))
    return tags_ordenadas[0][1] if tags_ordenadas else None

def extrair_titulo(title_wrapper):
    inst_back_texts = {}
    inst_back_elements = title_wrapper.find_all('div', class_='inst_back')
    for index, inst_back in enumerate(inst_back_elements):
        b_tag = inst_back.find('b')
        if b_tag:
            inst_back_texts[index] = b_tag.text.strip()
    return inst_back_texts if inst_back_texts else None

def extrair_indices(layout_celula):
    indice = layout_celula.select_one('div.layout-cell-pad-5.text-align-right')
    valor = layout_celula.select_one('div.layout-cell.layout-cell-9 div.layout-cell-pad-5')
    if indice and valor:
        valores_extraidos = valor.text.split('<br class="clear">') if '<br class="clear">' in valor.text else valor.text.split('\n\t\t\n\t')
        return indice.text, valores_extraidos
    return None, None

def imprimir_informacoes(dados_json, nome_no, indent=0):
    indentation = '    ' * indent  # Calculating the current indentation level

    if dados_json and nome_no and dados_json.get(nome_no):
        if indent == 0:  # Logging node-level information only at the root
            logging.info(f"{indentation}Node: {nome_no}")
            logging.info(f"{indentation}Total keys extracted: {len(dados_json[nome_no].keys())}")
        
        for key in dados_json[nome_no].keys():
            logging.info(f"{indentation}{key.strip() if key else ''}")

            if isinstance(dados_json[nome_no][key], dict):  # Check for nested dictionaries
                # Recursive call to handle nested dictionaries
                imprimir_informacoes(dados_json[nome_no], key, indent + 1)
            else:
                logging.info(f"{indentation}    Values: {dados_json[nome_no][key]}")

In [ ]:
# for i in soup.select('div.layout-cell.layout-cell-12.data-cell'):
#     print(i.text.strip())

In [ ]:
from bs4 import BeautifulSoup
import json

def extrair_wraper(soup, json_data):
    title_wrappers = soup.select('div.layout-cell-pad-main div.title-wrapper')
    for title_wrapper in title_wrappers:
        section_name = extrair_secao(title_wrapper)
        if section_name:
            section_name = section_name.text.strip()
            
            titles = extrair_titulo(title_wrapper)
            json_data["Properties"][section_name] = {}
            
            if titles:
                for index, title in titles.items():
                    json_data["Properties"][section_name][title] = {}
            
            layout_cells = title_wrapper.select('div.layout-cell.layout-cell-12.data-cell')
            for layout_celula in layout_cells:
                indice, valores_extraidos = extrair_indices(layout_celula)
                if indice and valores_extraidos:
                    if titles and indice in titles.values():
                        if len(titles) > 1:
                            for title in titles.values():
                                if title.strip() in indice:
                                    json_data["Properties"][section_name][title][indice] = valores_extraidos
                        else:
                            title = list(titles.values())[0]
                            json_data["Properties"][section_name][title][indice] = valores_extraidos
                    else:
                        json_data["Properties"][section_name][indice] = valores_extraidos
    return json_data

def extrair_dados(soup, verbose=False):
    # Step 1: Identify Node Name
    node_name_element = soup.select_one('div.infpessoa h2.nome')
    node_name = node_name_element.text if node_name_element else None
    
    if verbose:
        print(f"Node name: {node_name}")

    # Step 2: Initialize JSON object
    json_data = {"Node Name": node_name, "Properties": {}}
    
    # Step 3: Traverse to Find Sections
    json_data = extrair_wraper(soup, json_data)

    # Step 4: Information extraction: Bibliographic Production Section
    producao_bibliografica_div = soup.find('div', {'id': 'artigos-completos'})
    producoes = []
    for artigo_div in producao_bibliografica_div.find_all('div', {'class': 'artigo-completo'}):
        artigo_dict = {}

        # Extrair os números de ordem dos artigos
        ordens = artigo_div.find_all('div', {'class': 'layout-cell-pad-5 text-align-right'})
        for index, ordem in enumerate(ordens):
            b_tag = ordem.find('b')
            # if b_tag:
            #     print(b_tag.text.strip())
            
        try:
            ano = artigo_div.find('span', {'data-tipo-ordenacao': 'ano'}).text
        except:
            ano = None
        try:
            prim_autor = artigo_div.find('span', {'data-tipo-ordenacao': 'autor'}).text
        except:
            prim_autor = None
        try:
            jcr = artigo_div.find('span', {'data-tipo-ordenacao': 'jcr'}).text
        except:
            jcr = None
        try:
            doi = artigo_div.find('a', {'class': 'icone-doi'})['href']
        except:
            doi = None
        # try:
        #     titulo = artigo_div.find('div', {'class': 'citado'}).text
        # except:
        #     titulo = None
        dados   = artigo_div.find_all('div', {'class': 'layout-cell-pad-5'})
        list    = str(dados).split(" . ")
        autores = prim_autor + list[0].split(prim_autor)[-1].replace('</a>','').replace('</b>','').replace('<b>','')
        revista = list[1].split('nomePeriodico=')[1].split('tooltip=')[0].strip('\" ')
        titulo  = list[1].split('titulo=')[1].split('&amp')[0].strip('\" ')
        artigo_dict['ano']     = ano
        artigo_dict['autores'] = autores
        artigo_dict['revista'] = revista
        artigo_dict['titulo']  = titulo
        artigo_dict['jcr']     = jcr
        artigo_dict['doi']     = doi
        producoes.append(artigo_dict)
        json_data["Properties"]['Produções'] = producoes
    
    return json_data

## Extração do objeto Soup do Lattes em HTML

In [ ]:
driver = connect_driver()
NOME = ['Antonio Marcos Aires Barbosa']
fill_name(driver, delay, NOME)

limite=3
instituicao = 'Fundação Oswaldo Cruz'
unidade     = 'Fiocruz Ceará'
termo       = 'Ministerio da Saude'

elm_vinculo, np.NaN, np.NaN, np.NaN, driver = find_terms(NOME, instituicao, unidade, termo, driver, delay, limite)
soup = open_curriculum(driver,elm_vinculo)

caracteres = len(soup.text)
linhas = len(soup.text.split('\n'))
print(f'Total de caracteres extraídos: {caracteres:6d}')
print(f'Quantidade extraída de linhas: {linhas:6d}')

In [ ]:
json_data = extrair_dados(soup, True)
nome_no = 'Antonio Marcos Aires Barbosa'
imprimir_informacoes({nome_no: json_data['Properties']}, nome_no)

In [ ]:
json_data

In [ ]:
layout_cell_main = soup.select_one('div.layout-cell-pad-main')
for title_wrapper in layout_cell_main.find_all('div.title-wrapper'):
    index = title_wrapper.select_one('ul, a, b').text.strip()
    print(index)

In [ ]:
from bs4 import BeautifulSoup
import json

def generate_json_from_html(soup):
    """
    Generate a JSON object from an Soup object for a Neo4j integration.

    Parameters:
        soup (object): Soup object from a HTML content.

    Returns:
        json_data (dict): A dictionary representing the JSON structure.
    """

    # Extract node name
    node_name = soup.select_one('div.infpessoa h2.nome').text.strip()

    # Initialize the master dictionary
    json_data = {node_name: {}}

    # Locate the main layout cell
    layout_cell_main = soup.select_one('div.layout-cell-pad-main')

    # Iterate over title-wrapper elements
    for title_wrapper in layout_cell_main.select('div.title-wrapper'):
        index = title_wrapper.select_one('ul, a, b').text.strip()
        
        # Initialize the child dictionary
        json_data[node_name][index] = {}

        # Iterate over layout cells
        for layout_cell_3 in title_wrapper.select('div.layout-cell.layout-cell-3.text-align-right'):
            grandchild_index = layout_cell_3.select_one('div.layout-cell-pad-5.text-align-right').text.strip()
            
            # Find the corresponding layout cell for values
            layout_cell_9 = layout_cell_3.find_next_sibling('div', class_='layout-cell.layout-cell-9')
            
            values_text = layout_cell_9.select_one('div.layout-cell-pad-5').text
            
            # Create a list of values
            values = values_text.split('<br class="clear">' or '\n\n\n')

            # Add the grandchild dictionary
            json_data[node_name][index][grandchild_index] = values

    return json_data

# Generate JSON data
json_data = generate_json_from_html(soup)

# Print or persist the generated JSON data
print(json.dumps(json_data, indent=4))

In [ ]:
json_lattes = extract_data(soup)

In [ ]:

pprint(json_lattes)

In [ ]:
import pandas as pd

def dict_to_dataframe(json_data, parent_key='', separator='.'):
    """
    Convert a nested dictionary into a flat dictionary, suitable for DataFrame conversion.
    
    Parameters:
    - json_data (dict): The nested dictionary to flatten.
    - parent_key (str, optional): The concatenated key used to represent nesting.
    - separator (str, optional): The character to use for separating nested keys.
    
    Returns:
    - pd.DataFrame: A DataFrame representing the flattened dictionary.
    """
    flat_dict = {}
    
    for k, v in json_data.items():
        new_key = f"{parent_key}{separator}{k}" if parent_key else k
        
        if isinstance(v, dict):
            flat_dict.update(dict_to_dataframe(v, new_key, separator=separator))
        else:
            flat_dict[new_key] = v
            
    return pd.DataFrame([flat_dict])

In [ ]:
# Assuming json_lattes is a JSON-formatted string
json_lattes_dict = json.loads(json_lattes)

# Then call the dict_to_dataframe function
df = dict_to_dataframe(json_lattes_dict)
df.T

In [ ]:
# Extração com BeaultifulSoap
def parse_header(soup):
    splpt = 'Currículo do Sistema de Currículos Lattes ('
    string_title = soup.title.string if soup.title else "Unknown"
    title = string_title.split(splpt)[1].strip(')')
    
    meta_keywords = soup.find("meta", {"http-equiv": "keywords"})["content"] if soup.find("meta", {"http-equiv": "keywords"}) else "Unknown"
    return {"title": title, "meta_keywords": meta_keywords}

def parse_h1_elements(soup, parent_node, graph):
    h1_elements = soup.find_all('h1', {'tabindex': '0'})
    secoes = []
    
    print(f'{len(h1_elements[2:])} elementos encontrados')
    for n,i in enumerate(h1_elements):
        if n>1:
            secao = i.text
            print(f'    {secao}')
            secoes.append(secao)
    
    for elem in h1_elements[2:]:
        h1_text = elem.text.strip()
        h1_node = Node("H1Element", text=h1_text)
        
        graph.create(h1_node)
        relation = Relationship(parent_node, "HAS_SECTION", h1_node)
        graph.create(relation)

In [ ]:
def parse_parsoninfo(soup):
    # Localizar o elemento de link que contém o título do currículo
    link_element = soup.find("a", {"href": lambda x: x and "abreDetalhe" in x})

    # Extrair o texto do link para usar como título do nó
    node_title = link_element.text if link_element else "Unknown"
    print(f'Título do Nó: {node_title}')

    # Localizar o elemento div contendo as propriedades
    properties_div = soup.find("div", {"class": "resultado"})
    if properties_div:
        print(f'Resultado: {properties_div.text}')
    else:
        print('Resultados não encontrados')

    # Inicializar um dicionário para armazenar as propriedades
    properties = {}
    
    # Localizar o elemento li que contém as informações do idlattes
    li_element = soup.find("li")
    for i in li_element:
        if 'http://lattes.cnpq.br/' in i:
            idlattes = i.split('http://lattes.cnpq.br/')[1]
            properties['Idlattes'] = idlattes
            print(idlattes)

    # Extrair e armazenar as propriedades relevantes
    if properties_div:
        properties['Nacionalidade'] = 'Brasil'
        properties['Cargo'] = properties_div.find("span", {"class": "tooltip coautor"}).text if properties_div.find("span", {"class": "tooltip coautor"}) else 'Desconhecido'
        properties['Titulação'] = properties_div.contents[-4] if len(properties_div.contents) > 4 else 'Desconhecido'

        # Extração de nome e identificador único
        a_element = li_element.find("a")
        properties["Nome"] = a_element.text

        # Extração de Nacionalidade
        img_element = li_element.find("img")
        properties["Nacionalidade"] = img_element.get("title") or img_element.get("alt")

        # Extração de Bolsa de Produtividade
        span_element = li_element.find("span", {"class": "tooltip coautor"})
        properties["Bolsista_Prod_Pesquisa"] = span_element.get("alt")

        # Extração de Nacionalidade
        img_element = li_element.find("img")
        properties["Nacionalidade"] = img_element.get("title") or img_element.get("alt")

        # Extração de Bolsa de Produtividade
        span_element = li_element.find("span", {"class": "tooltip coautor"})
        properties["Bolsista_Prod_Pesquisa"] = span_element.get("alt")

        # Extração de Títulos Acadêmicos e outras informações

    return node_title, properties

In [ ]:
def extract_infopessoa(soup):
    # Localiza a seção com a classe 'infpessoa'
    section = soup.find('div', class_='infpessoa')

    # Inicializa um dicionário para armazenar os dados extraídos
    extracted_data = {}

    # Extrai e armazena o nome
    name = section.find('h2', class_='nome').text.strip() if section.find('h2', class_='nome') else None
    extracted_data['Nome'] = re.sub(r'\s+', ' ', name) if name else None

    # Extrai e armazena o título ou posição
    title = section.find('span', class_='texto').text.strip() if section.find('span', class_='texto') else None
    extracted_data['Bolsa'] = re.sub(r'\s+', ' ', title) if title else None

    # Extrai e armazena as informações adicionais
    info_list = section.find('ul', class_='informacoes-autor')
    if info_list:
        for li in info_list.find_all('li'):
            text = re.sub(r'\s+', ' ', li.text.strip())
            if 'Endereço para acessar este CV:' in text:
                extracted_data['Endereço para acessar este CV'] = text.replace('Endereço para acessar este CV:', '').strip()
            elif 'ID Lattes:' in text:
                extracted_data['ID Lattes'] = text.replace('ID Lattes:', '').strip()
            elif 'Última atualização do currículo em' in text:
                extracted_data['Última atualização do currículo em'] = text.replace('Última atualização do currículo em', '').strip()

    extracted_data['Resumo'] = soup.find('p', class_='resumo').text.strip()
    
    return extracted_data


def extract_academic(soup):
    """
    Extrai dados da página do Lattes, nas divs com classes 'layout-cell-pad-5 text-align-right' e 'layout-cell-pad-5' em um dicionário.
    
    Parâmetros:
    - Bs4 object (soup): Objeto dod Beaultiful Soap com o conteúdo bruto da página HTML.
    
    Retorno:
    - Um dicionário contendo os pares de chave-valor extraídos.
    """
    # Dicionário para armazenar os pares de chave-valor extraídos
    extracted_data = {}
    
    # Localiza todas as divs com a classe 'title-wrapper'
    divs_key = soup.find_all('div', {'class': 'title-wrapper'})
    
    
    for div_key in divs_key:
        # Extrai o conteúdo da tag para formar a chave do dicionário
        find_div = div_key.find_next_sibling('div').find('div', {'class': 'layout-cell-pad-5 text-align-right'})
        if find_div:
            key = find_div.text.strip('\n')
        
        # Encontra a div que segue imediatamente para o valor
        div_value = div_key.find_next_sibling('div').find('div', {'class': 'layout-cell layout-cell-9'})
        
        # Extrai o conteúdo da div para o valor
        value = div_value.text.strip() if div_value else None
        
        # Armazena no dicionário se ambas chave e valor existirem
        if key and value:
            extracted_data[key] = value
    
    return extracted_data


def mount_articles(extracted_content):
    """
    Organiza os dados extraídos pelo web scraping em um DataFrame do pandas, melhorando a detecção de '\n'.
    
    Parâmetros:
    - extracted_content (dict): Um dicionário contendo os conteúdos extraídos organizados por classe.
    
    Retorno:
    - Um DataFrame do pandas contendo os dados organizados.
    """
    # Localiza a lista de artigos completos
    artigos_completos = extracted_content.get('artigo-completo', [])
    print(f'Total de artigos detectados: {len(artigos_completos)}')
    
    # Concatena todos os artigos em uma única string
    artigos_concatenados = " ".join(artigos_completos)
    
    # Remove múltiplos espaços e substitui por um único espaço
    artigos_concatenados = re.sub(r'\s+', ' ', artigos_concatenados)
    
    # Divide os artigos com base em um padrão específico (exemplo: números seguidos de pontos)
    artigos_divididos = re.split(r'\s\d+\.\s', artigos_concatenados)
    
    # Remove entradas vazias
    artigos_divididos = [artigo.strip() for artigo in artigos_divididos if artigo.strip()]
    
    # Lista para armazenar os registros para o DataFrame
    records = []
    ordens = []
    
    for artigo in artigos_divididos:
        # Encontra o primeiro ano mencionado no artigo
        match_ano = re.search(r'20\d{2}', artigo)
        
        if match_ano:
            ano = match_ano.group(0)
            indice, resto = artigo.split(ano, 1)
            
            # Armazena o registro em formato de dicionário
            record = {
                'Índice': indice.split('. ')[0],
                'Ano': ano,
                'Título e Autores': resto.strip()
            }
            records.append(record)
    
    # Cria um DataFrame do pandas com os registros
    df = pd.DataFrame(records)
    
    return df


def extract_research_project(soup):
    project_list = []
    projects_section = soup.find('h1', string='Projetos de pesquisa')
    periods=[]
    titles=[]
    descriptions=[]
    if projects_section:
        project_divs = projects_section.find_all_next('div', {'class': 'layout-cell layout-cell-3 text-align-right'})
        
        for div in project_divs:
            project_dict = {}
            period_div = div.find('b')
            if period_div:
                periods.append(period_div.text.strip())
            
            title_div_container = div.find_next_sibling('div')
            if title_div_container:
                title_div = title_div_container.find('div', {'class': 'layout-cell-pad-5'})
                if title_div:
                    titles.append(title_div.text.strip())
            
            # Locate the div that contains the project description
            parent_div = div.find_parent('div')
            if parent_div:
                description_div_container = parent_div.find_next_sibling('div')
                if description_div_container:
                    description_div_container = description_div_container.find_next_sibling('div')
                    if description_div_container:
                        description_div = description_div_container.find('div', {'class': 'layout-cell-pad-5'})
                        if description_div:
                            full_text = description_div.text
                            description_start_index = full_text.find('Descrição:')
                            if description_start_index != -1:
                                descriptions.append(full_text[description_start_index + len('Descrição:'):].strip())

    df =pd.DataFrame({
        'PERIODO': pd.Series(periods),
        'TITULO': pd.Series(titles),
        'DESCRICAO': pd.Series(descriptions),
            })                
    
    descricoes = df[df['PERIODO']==""]['TITULO'].values
    df = df[df['PERIODO']!=""]
    df = df[:len(descricoes)]
    df['DESCRICAO']=descricoes
    df
    
    return df

In [ ]:
def store_annotation_in_db(uri, user, password, annot_html, cv_id):
    """
    Store the annotation HTML in a Neo4j database.

    Parameters:
        uri (str): URI of the Neo4j database
        user (str): Username for the Neo4j database
        password (str): Password for the Neo4j database
        annot_html (str): The HTML string containing annotations
        cv_id (str): The unique identifier for the annotated CV

    Returns:
        None
    """
    # Initialize Neo4j driver
    driver = GraphDatabase.driver(uri, auth=(user, password))

    # Define Cypher query for adding an annotation
    add_annotation_query = '''
    MERGE (cv:CV {id: $cv_id})
    CREATE (a:Annotation {html: $annot_html})
    MERGE (cv)-[:HAS]->(a)
    '''

    # Execute query
    with driver.session() as session:
        session.run(add_annotation_query, cv_id=cv_id, annot_html=annot_html)

In [ ]:
# from bs4 import BeautifulSoup
# from collections import Counter

# def enumerate_tags(soup):
#     # Extração de todos os marcadores (tags) no documento
#     all_tags = [tag.name for tag in soup.find_all(True)]
    
#     # Contagem de ocorrências de cada marcador
#     tag_count = Counter(all_tags)
    
#     # Conversão do objeto Counter para um dicionário padrão para facilitar a manipulação subsequente
#     tag_dict = dict(tag_count)
    
#     return tag_dict

# # Exemplo de uso
# tag_dictionary = enumerate_tags(soup)
# print(tag_dictionary)

In [ ]:
# div_elements = soup.find_all('div')
# div_elements

In [ ]:
# from bs4 import BeautifulSoup

# def extract_div_data(soup):
#     """
#     Extrai dados das divs com classes 'layout-cell-pad-5 text-align-right' e 'layout-cell-pad-5' em um dicionário.
    
#     Parâmetros:
#     - html_document (str): String contendo o documento HTML.
    
#     Retorno:
#     - Um dicionário contendo os pares de chave-valor extraídos.
#     """
#     # Dicionário para armazenar os pares de chave-valor extraídos
#     extracted_data = {}
    
#     # Localiza todas as divs com a classe 'layout-cell-pad-5 text-align-right'
# #     divs_key = soup.find_all('div', {'class': 'layout-cell-pad-5 text-align-right'})
#     divs_key = soup.find_all('div', {'class': 'title-wrapper'})
    
    
#     for div_key in divs_key:
#         # Extrai o conteúdo da tag <b> dentro da div
#         key = div_key.find('b').text if div_key.find('b') else None
        
#         # Encontra a div que segue imediatamente
#         div_value = div_key.find_next_sibling('div').find('div', {'class': 'layout-cell-pad-5'})
        
#         # Extrai o conteúdo da div
#         value = div_value.text.strip() if div_value else None
        
#         # Armazena no dicionário se ambas chave e valor existirem
#         if key and value:
#             extracted_data[key] = value
    
#     return extracted_data

In [ ]:
# extract_div_data(soup)

In [ ]:
# pprint(extracted_content)

## Refinamento dos dados

In [ ]:
def list_divs(soup):
    div_elements = soup.find_all('div')
    unique_classes = set()

    for div in div_elements[1:]:
        div_id = div.get('id', 'N/A')
        class_list = div.get('class')
        print(f'{div_id} {class_list}')
        
        if class_list:
            for i in class_list:
                cont = div.find('div',{'class': i})
                try:
                    text = cont.text
                    print(f'  {text}')
                except:
                    pass

            print('-'*50)
            unique_classes.update(class_list)
                
    print('\nLista de Classes únicas:')
    for i in unique_classes:
        print(f'  {i}')

In [ ]:
def count_classes(soup): 
    # Extração de todas as classes no documento, armazenadas em uma lista
    all_classes = [value for element in soup.find_all(class_=True) for value in element["class"]]
    
    # Contagem de ocorrências de cada classe
    class_count = Counter(all_classes)
    
    # Conversão do objeto Counter para um dicionário padrão para facilitar a manipulação subsequente
    class_dict = dict(class_count)
    
    return class_dict

In [ ]:
list_divs(soup)

In [ ]:
count_classes(soup)

In [ ]:
def list_divs_with_hierarchy(tag, indent_level=0):
    indent = " " * indent_level * 4  # Four spaces for each level of indentation
    div_elements = tag.find_all('div', recursive=False)  # Find direct children only
    unique_classes = set()

#     print(f"{indent}Inspecting level {indent_level}, found {len(div_elements)} divs.")  # Debugging line

    for div in div_elements:
        class_list = div.get('class')
        if class_list:
            unique_classes.update(class_list)
            print(f"{indent} {', '.join(class_list)}")
        else:
            print(f"{indent} None")

        # Recursive call to explore the children of this div
        list_divs_with_hierarchy(div, indent_level + 1)

    if indent_level == 0:  # Print unique classes only once, at the end of the initial call
        print("\nUnique classes:")
        for i in unique_classes:
            print(f'    {i}')

# # Initialize BeautifulSoup with a sample HTML content
# soup_sample = BeautifulSoup('<html><head></head><body><div class="test">Hello</div></body></html>', 'html.parser')
# print(soup_sample.prettify())

# # Start from the <body> tag
# body_tag = soup_sample.body

# # Then try calling list_divs_with_hierarchy on the <body> tag
# list_divs_with_hierarchy(body_tag)

In [ ]:
from bs4 import BeautifulSoup

def extract_data(soup):
    result_dict = {}
    
    title_elements = soup.find_all('h1')
    for title_elem in title_elements:
        title_text = title_elem.text.strip()
        
        data_dict = {}
        
        parent_div = title_elem.find_parent('div', {'class': 'title-wrapper'})
        if parent_div:
            data_cells = parent_div.find_all('div', {'class': 'layout-cell layout-cell-12 data-cell'})
            
            higher_order_key = None  # Initialize a variable to store the higher-order key
            data_list = []  # Initialize a list to store year-details entries
            
            for cell in data_cells:
                inst_back_elem = cell.find('div', {'class': 'inst_back'})
                
                if inst_back_elem:
                    higher_order_key = inst_back_elem.text.strip()
                    data_list = []  # Reset the list for the new higher-order key
                    data_dict[higher_order_key] = data_list  # Create a new list for this higher-order key
                    
                year_elems = cell.find_all('div', {'class': 'layout-cell-pad-5 text-align-right'})
                details_elems = cell.find_all('div', {'class': 'layout-cell layout-cell-9'})
                
                for year_elem, details_elem in zip(year_elems, details_elems):
                    year_text = year_elem.text.strip() if year_elem else None
                    details_text = details_elem.text.strip() if details_elem else None
                    
                    data_entry = {'rotulos': year_text, 'conteudos': details_text}
                    
                    data_list.append(data_entry)  # Append each entry to the list
                
            if higher_order_key is None:
                # If no higher-order key is found, associate the data list directly with the title
                result_dict[title_text] = data_list
            else:
                # Otherwise, associate the data_dict containing higher-order keys with the title
                result_dict[title_text] = data_dict

    return result_dict

In [ ]:
data_dict = extract_data(soup)
data_dict

In [ ]:
import pandas as pd

def generate_dataframe(data_dict):
    frames = {}  # Store DataFrames corresponding to each section
    for section, items in data_dict.items():
        if type(items) is list:
            df = pd.DataFrame(items)
            df['Section'] = section  # Add a column to identify the section
            frames[section] = df
        elif type(items) is dict:
            for subsection, subitems in items.items():
                df = pd.DataFrame(subitems)
                df['Subsection'] = subsection  # Add a column to identify the subsection
                df['Section'] = section  # Add a column to identify the section
                frames[f"{section}_{subsection}"] = df
    return frames

frames = generate_dataframe(data_dict)

In [ ]:
list(frames.keys())

In [ ]:
# Access individual DataFrames like so:
identificacao_df = frames['Identificação']
identificacao_df

In [ ]:
endereco_df = frames['Endereço']
endereco_df

In [ ]:
formacao_df = frames['Formação acadêmica/titulação']
formacao_df

In [ ]:
formcompl_df = frames['Formação Complementar']
formcompl_df

In [ ]:
atuaprof_df = frames['Atuação Profissional_Fundação Oswaldo Cruz, FIOCRUZ, Brasil.']
atuaprof_df

In [ ]:
linhas_df = frames['Linhas de pesquisa']
linhas_df

In [ ]:
projdesv_df = frames['Projetos de desenvolvimento']
projdesv_df

In [ ]:
projpesq_df = frames['Áreas de atuação']
projpesq_df

In [ ]:
areasatuacao_df = frames['Áreas de atuação']
areasatuacao_df

In [ ]:
from bs4 import BeautifulSoup

def extract_data(soup):
    result_dict = {}
    
    title_elements = soup.find_all('h1')
    for title_elem in title_elements:
        title_text = title_elem.text.strip()
        
        data_dict = {}
        
        parent_div = title_elem.find_parent('div', {'class': 'title-wrapper'})
        if parent_div:
            data_cells = parent_div.find_all('div', {'class': 'layout-cell layout-cell-12 data-cell'})
            
            current_higher_order_dict = None  # Initialize a variable to store the current higher-order dictionary
            
            for cell in data_cells:
                inst_back_elem = cell.find('div', {'class': 'inst_back'})
                
                if inst_back_elem:
                    higher_order_key = inst_back_elem.text.strip()
                    current_higher_order_dict = {}  # Create a new dictionary for this higher-order key
                    data_dict[higher_order_key] = current_higher_order_dict  # Associate the new dictionary with the higher-order key
                    
                year_elems = cell.find_all('div', {'class': 'layout-cell-pad-5 text-align-right'})
                details_elems = cell.find_all('div', {'class': 'layout-cell layout-cell-9'})
                
                for year_elem, details_elem in zip(year_elems, details_elems):
                    year_text = year_elem.text.strip() if year_elem else None
                    details_text = details_elem.text.strip() if details_elem else None
                    
                    if current_higher_order_dict is not None:
                        # Insert the year-details pair into the current higher-order dictionary
                        current_higher_order_dict[year_text] = details_text
                    else:
                        # If no higher-order key is present, associate the year-details pair directly with the title
                        data_dict[year_text] = details_text
                
            result_dict[title_text] = data_dict

    return result_dict

In [ ]:
import pandas as pd

def dict_to_dataframe(input_dict):
    def recursive_descent(current_dict, parent_key='', separator='.'):
        nonlocal flattened_dict
        for k, v in current_dict.items():
            new_key = f"{parent_key}{separator}{k}" if parent_key else k
            if isinstance(v, dict):
                recursive_descent(v, new_key, separator=separator)
            else:
                flattened_dict[new_key] = v
                
    flattened_dict = {}
    recursive_descent(input_dict)
    
    # Create DataFrame from the flattened dictionary
    df = pd.DataFrame([flattened_dict])
    
    return df

In [ ]:
extract_data(soup)

In [ ]:
nested_dict = extract_data(soup)
df = dict_to_dataframe(nested_dict)
df.T

In [ ]:
from bs4 import BeautifulSoup
from bs4.element import Tag, NavigableString

html_content = '''
<div class="layout-cell-pad-5">
    Doutorado em andamento em Informática Aplicada.
    <br class="clear">Universidade de Fortaleza, UNIFOR, Brasil.
    <br class="clear">Título: Processamento de Linguagem Natural no Desenvolvimento de Bioprodutos para Saúde
    <br class="clear">Orientador: <a class="icone-lattes" target="_blank" href="http://lattes.cnpq.br/2607811863279622" tabindex="80"><img src="images/curriculo/logolattes.gif"></a>Raimir Holanda Filho.
</div>
'''

soup_sample = BeautifulSoup(html_content, 'html.parser')

def extract_text_by_br_class(soup):
    result_list = []
    elements = soup.select('div.layout-cell-pad-5')
    for element in elements:
        text_segments = []
        current_dict = {}
        for content in element.contents:
            if isinstance(content, Tag):
                if content.get('class') == ['clear']:
                    joined_text = ' '.join(text_segments).strip()
                    if "Orientador:" in joined_text:
                        key, value = joined_text.split("Orientador:")
                        current_dict[key.strip()] = {'name': value.strip(), 'url': current_dict.pop('url', None)}
                    else:
                        current_dict[joined_text] = None
                    text_segments = []
                elif content.name == 'a':
                    current_dict['url'] = content.get('href')
            elif isinstance(content, NavigableString):
                text_segments.append(str(content).strip())
        # Capture any remaining text
        if text_segments:
            joined_text = ' '.join(text_segments).strip()
            current_dict[joined_text] = None
        result_list.append(current_dict)
    return result_list

# Execução da função
extracted_texts = extract_text_by_br_class(soup_sample)
extracted_texts

In [ ]:
from bs4 import BeautifulSoup
def extract_text_from_selectors(soup,select_path):
    result_list = []
    
    # Identificando os elementos que correspondem aos seletores desejados.
    for title_wrapper in soup.select('div.title-wrapper'):
        elements = title_wrapper.select(select_path)
#         print(len(elements))
        
        # Extração do texto dos elementos identificados.
        for element in elements:
            text_content = element.text.strip()
            result_list.append(text_content)
            
    return result_list

In [ ]:
from bs4.element import Tag, NavigableString
def extract_text_by_br_class(soup):
    result_list = []
    elements = soup.select('div.layout-cell-pad-5')
    for element in elements:
        text_segments = []
        current_dict = {}
        for content in element.contents:
            if isinstance(content, Tag):
                if content.get('class') == ['clear']:
                    joined_text = ' '.join(text_segments).strip()
                    if "Orientador:" in joined_text:
                        key, value = joined_text.split("Orientador:")
                        current_dict[key.strip()] = {'name': value.strip(), 'url': current_dict.pop('url', None)}
                    else:
                        current_dict[joined_text] = None
                    text_segments = []
                elif content.name == 'a':
                    current_dict['url'] = content.get('href')
            elif isinstance(content, NavigableString):
                text_segments.append(str(content).strip())
        # Capture any remaining text
        if text_segments:
            joined_text = ' '.join(text_segments).strip()
            current_dict[joined_text] = None
        result_list.append(current_dict)
    return result_list

# Execução da função
extracted_texts = extract_text_by_br_class(soup)

In [ ]:
extracted_texts

In [ ]:
keys="layout-cell.layout-cell-3.text-align-right"
vals="layout-cell layout-cell-9"
<div class="layout-cell layout-cell-9">
<div class="layout-cell-pad-5">Doutorado em andamento em Informática Aplicada. <br class="clear">Universidade de Fortaleza, UNIFOR, Brasil.
		
	<br class="clear">Título: Processamento de Linguagem Natural no Desenvolvimento de Bioprodutos para Saúde<br class="clear">Orientador: <a class="icone-lattes" target="_blank" href="http://lattes.cnpq.br/2607811863279622" tabindex="80"><img src="images/curriculo/logolattes.gif"></a>Raimir Holanda Filho. </div>
</div>

from bs4 import BeautifulSoup
father     = 'div.title-wrapper'
sons       = 'h1'
grandchild = '' 
def extract_text_titles(soup,father,sons):
    result_list = []
    
    # Identificando os elementos que correspondem aos seletores desejados.
    for father_element in soup.select(father):
        son_elements = father_element.select(sons)
        
        # Extração do texto dos elementos identificados.
        for grandchild in son_elements:
            text_content = element.text.strip()
            result_list.append(text_content)
            
    return result_list

In [ ]:
extract_text_titles(soup,father,sons)

In [ ]:
from bs4 import BeautifulSoup
father = 'div.title-wrapper'
sons   = 'h1'
def extract_text_titles(soup,father,sons):
    result_list = []
    
    # Identificando os elementos que correspondem aos seletores desejados.
    for father_element in soup.select(father):
        elements = father_element.select(sons)
        
        # Extração do texto dos elementos identificados.
        for element in elements:
            text_content = element.text.strip()
            result_list.append(text_content)
            
    return result_list

In [ ]:
# Execução da função
# <div class="layout-cell-pad-5 text-align-right">
# <b>2017 - 2019</b>
# </div>
select_path='div.layout-cell-pad-12.data-cell'
result_list = extract_text_from_selectors(soup,select_path)
pprint(result_list)

In [ ]:
# Execução da função
# <div class="layout-cell-pad-5 text-align-right">
# <b>2017 - 2019</b>
# </div>
select_path='div.layout-cell-pad-5.text-align-right'
result_list = extract_text_from_selectors(soup,select_path)
pprint(result_list)

In [ ]:
# Execução da função
select_path='layout-cell-pad-5.text-align-right'
result_list = extract_text_from_selectors(soup,select_path)
pprint(result_list)

In [ ]:
from bs4 import BeautifulSoup
from collections import defaultdict

def extract_to_dict(soup):
    result_list = []
    
    for title_wrapper in soup.select('div.title-wrapper'):
        temp_dict = defaultdict(dict)
        first_level_key = title_wrapper.select_one('div.layout-cell.layout-cell-12.data-cell').text.strip()
        
        for cell in title_wrapper.select('div.layout-cell.layout-cell-12.data-cell'):
            second_level_keys = cell.select('div.layout-cell-pad-5.text-align-right')
            values = cell.select('div.layout-cell.layout-cell-3.text-align-right')
            
            if len(second_level_keys) == len(values):
                for key, value in zip(second_level_keys, values):
                    second_level_key = key.text.strip()
                    value_text = value.text.strip()
                    temp_dict[first_level_key][second_level_key] = value_text
        
        result_list.append(temp_dict)
    
    return result_list

# Execute the function
result_list = extract_to_dict(soup)

In [ ]:
l = ['layout-cell', 'layout-cell-3', 'text-align-right']

# Start from the <body> tag
body_tag = soup.body
# 'text-align-right': 152,
# 'layout-cell-pad-5': 152,

content_wrapper_tag = body_tag.find('div',
                                    {'class': 'layout-cell-pad-main'},
                                   )

list_divs_with_hierarchy(content_wrapper_tag, indent_level=0)

In [ ]:
# Start from the <body> tag
body_tag = soup.body
content_wrapper_tag = body_tag.find('div',
                                    {'class': 'layout-cell'},
                                   )

list_divs_with_hierarchy(content_wrapper_tag, indent_level=0)

In [ ]:
# Start from the <body> tag
body_tag = soup.body
content_wrapper_tag = body_tag.find('div',
                                    {'class': 'text-align-right'},
                                   )

list_divs_with_hierarchy(content_wrapper_tag, indent_level=0)

In [ ]:
# Start from the <body> tag
body_tag = soup.body
content_wrapper_tag = body_tag.find('div',
                                    {'class': 'layout-cell-9'},
                                   )

list_divs_with_hierarchy(content_wrapper_tag, indent_level=0)

In [ ]:
# Start from the <body> tag
body_tag = soup.body
content_wrapper_tag = body_tag.find('div',
                                    {'class': 'layout-cell-3'},
                                   )

list_divs_with_hierarchy(content_wrapper_tag, indent_level=0)

In [ ]:
# Start from the <body> tag
body_tag = soup.body
content_wrapper_tag = body_tag.find('div',
                                    {'class': 'data-cell'},
#                                     {'class': 'layout-cell'},
#                                     {'class': 'layout-cell-9},                                    
#                                     {'class': 'layout-cell-12'},
#                                     {'class': 'layout-cell-pad-5'},
#                                     {'class': 'data-cell'}
                                   )

list_divs_with_hierarchy(content_wrapper_tag, indent_level=0)

In [ ]:
from bs4 import BeautifulSoup
from collections import defaultdict

def extract_text_from_hierarchy(tag, result_dict, parent_key="root"):
    div_elements = tag.find_all('div', recursive=False)
    
    for div in div_elements:
        class_list = div.get('class')
        class_key = ', '.join(class_list) if class_list else "None"
        
        # Criando uma chave única que incorpora o caminho da raiz até esta div
        full_key = f"{parent_key} -> {class_key}"

        # Coleta o texto contido no elemento div atual
        text_content = div.get_text(strip=True)

        # Armazenar o conteúdo textual sob esta chave única
        if full_key not in result_dict:
            result_dict[full_key] = []
        result_dict[full_key].append(text_content)

        # Chamada recursiva para extrair textos dos filhos deste div
        extract_text_from_hierarchy(div, result_dict, full_key)

In [ ]:
result_dict = defaultdict(list)

extract_text_from_hierarchy(soup.body, result_dict, 'text-align-right')

In [ ]:
for i in result_dict.keys():
    print(i,'\n')

In [ ]:
for i in result_dict.values():
    print(i,'\n')

In [ ]:
from collections import defaultdict
from bs4 import BeautifulSoup

def extract_key_value_pairs(soup_element, result_dict, key_classes, value_classes, key_stack=None):
    if key_stack is None:
        key_stack = []
    
    popped_key = None
    if soup_element.name == "div":
        class_list = soup_element.get('class', [])
        
        if any(k in class_list for k in key_classes):
            new_key = soup_element.text.strip()
#             print(f'Key found: {new_key}')  # Debugging line
            key_stack.append(new_key)
        
        elif any(v in class_list for v in value_classes) and key_stack:
            value = soup_element.text.strip()
#             print(f'Value found: {value}')  # Debugging line
            current_key = key_stack[-1]
            if current_key in result_dict:
                result_dict[current_key].append(value)
            else:
                result_dict[current_key] = [value]
                
#     print(f"Current key_stack: {key_stack}")  # Debugging line

    for child in soup_element.find_all("div", recursive=False):
        extract_key_value_pairs(child, result_dict, key_classes, value_classes, key_stack)

    if popped_key:
        key_stack.append(popped_key)

In [ ]:
soup_element  = soup.find('div', {'class': 'text-align-right'})
# key_classes   = ['data-cell']
# value_classes = ['text-align-right']

key_classes   = ['layout-cell-3']
value_classes = ['layout-cell-9']

extract_key_value_pairs(soup_element, result_dict, key_classes, value_classes)

In [ ]:
for i in result_dict.keys():
    pprint(i)

print()

for i in result_dict.values():
    pprint(i)

In [ ]:
soup_element  = soup.find('div', {'class': 'layout-cell-pad-5'})
key_classes   = ['layout-cell-3']
value_classes = ['layout-cell-9']

extract_key_value_pairs(soup_element, result_dict, key_classes, value_classes)
pprint(result_dict)

In [ ]:
classe = 'infpessoa'

def extract_target_classes(soup,classe):
    soup_element  = soup.find('div', {'class': classe})
#     key_classes   = ['data-cell']
#     value_classes = ['text-align-right']
    key_classes   = ['layout-cell-9']
    value_classes = ['layout-cell-pad-5']
    
    target_classes = [
        'Produção '
        'Bibliográfica',
        'Produção '
        'Técnica',
        'Produção '
        'Artística/Cultural'
        'nome', 
        'resumo', 
        'artigo-completo', 
        'cita', 
        'cita-artigos', 
        'citacoes', 
        'detalhes', 
        'fator', 
        'foto', 
        'informacao-artigo', 
        'informacoes-autor', 
        'infpessoa', 
        'rodape-cv', 
        'science_cont', 
        'texto', 
        'trab'
        ]
    try:
        extract_key_value_pairs(soup_element, result_dict, key_classes, value_classes)
        for x,y in zip(result_dict.keys(),result_dict.values()):
            if x in target_classes:
                try:
                    print(f"{x:>12} | {y[0]}")
                except:
                    pass
        
        filtered_dict = {k: v for k, v in result_dict.items() if k in target_classes}

    except:
        print(f'Classe "{classe}" não encontrada')
    return filtered_dict

In [ ]:
classe = 'infpessoa'
extract_target_classes(soup,classe)

In [ ]:
extract_target_classes(soup,classe)

In [ ]:
from collections import defaultdict
from bs4 import BeautifulSoup

def extract_classes_content(soup_element, target_classes):
    result_dict = defaultdict(list)
    
    for t_class in target_classes:
        for element in soup_element.find_all(class_=t_class):
            text_content = element.text.strip()
            result_dict[t_class].append(text_content)
            
    return dict(result_dict)

target_classes = [
        'artigo-completo', 'cita', 'cita-artigos', 'citacoes', 'detalhes', 'fator', 
        'foto', 'informacao-artigo', 'informacoes-autor', 'infpessoa', 'nome', 
        'resumo', 'rodape-cv', 'science_cont', 'texto', 'trab'
        ]

result_dict = extract_classes_content(soup.body, target_classes)
from pprint import pprint
pprint(result_dict)

In [ ]:
result_dict['informacao-artigo']

In [ ]:
for i in result_dict['artigo-completo']:
    print(i.split('\n\n\n\n'))

In [ ]:
target_classes = [
        'artigo-completo', 'cita', 'cita-artigos', 'citacoes', 'detalhes', 'fator', 
        'foto', 'informacao-artigo', 'informacoes-autor', 'infpessoa', 'nome', 
        'resumo', 'rodape-cv', 'science_cont', 'texto', 'trab'
        ]

def extract_selected_classes(soup, target_classes):
    """
    Extrai conteúdos de classes específicas de um objeto soup extraído de documento HTML.
    
    Parâmetros:
    - Objeto soup e lista de classes a serem extraídas.

    Retorno:
    - Um dicionário que mapeia o nome da classe à lista de conteúdos extraídos.
    """   
    
    # Dicionário para armazenar os conteúdos
    content_dict = defaultdict(list)
    
    # Iteração através das classes alvo para extração de conteúdo
    for target_class in target_classes:
        elements = soup.find_all(class_=target_class)
        for element in elements:
            content_dict[target_class].append(element.text.replace('\n\n\n','\n').replace('\n\n','\n').strip())
            
    return dict(content_dict)

In [ ]:
list_classes(soup)

In [ ]:
extract_infopessoa(soup)

In [ ]:
extract_academic(soup)

In [ ]:
df_id = pd.DataFrame([extract_academic(soup)])
df_id

In [ ]:
extracted_projects = extract_research_project(soup)
extracted_projects

In [ ]:
extract_selected_classes(soup)

In [ ]:
extracted_content = extract_selected_classes(soup)
mount_articles(extracted_content)

In [ ]:
# Extração e Persistência do cabeçalho
header_data = parse_header(soup)
header_data

In [ ]:
# dict_node = 
# header_node = persist_to_neo4j(header_data)

In [ ]:
# Extração e Persistência dos elementos H1
try:
    graph = Graph("bolt://localhost:7687", auth=("neo4j", "password"))
except Exception as e:
    print('Erro ao conectar ao Neo4j')
    print(e)
try:    
    header_node = persist_to_neo4j(header_data)
    print({type(header_node)})
    parse_h1_elements(soup, header_node, graph)
    cv_node, properties = parse_parsoninfo(soup)
except Exception as e:
    print('Erro ao persistir nó')
    print(e)

In [ ]:
graph

In [ ]:
cv_node

In [ ]:
properties

In [ ]:
parse_h1_elements(soup, parent_node, graph)

In [ ]:
header_data

In [ ]:
Node("Curriculum", 
     title=header_data['title'].split('(')[1].strip(')'), 
     meta_keywords=header_data['meta_keywords'])

In [ ]:
async def process_lattes_page(name_link, images_urls, recent_updates_url, qualis_data, data_source_info):
    print(qualis_data)
    
    # Inicializando o WebDriver
    driver_service = Service('path/to/chromedriver')
    driver = webdriver.Chrome(service=driver_service)
    driver.get(name_link['link'])
    
    # URL para visualização (pode ser determinado conforme a necessidade)
    visualization_url = 'path/to/visualization.html'
    
    # Limpar cache de dados de Qualis
    qualis_data_cache = {}
    
    # Annotate Lattes page com informações de Qualis
    lattes_info = await annotate_lattes_page(driver, images_urls, qualis_data, qualis_data_cache, data_source_info)
    
    if lattes_info:
        await inject_annotation_message(images_urls, visualization_url, recent_updates_url, len(lattes_info))
    
    # Consolidar dados de publicação a partir das informações de Lattes
    pub_info = consolidate_qualis_data(lattes_info)
    print(f"pubInfo: {pub_info}")
    
    # Tentativa de ler dados de Lattes do armazenamento local (neste caso, SQLite)
    conn = sqlite3.connect('lattes_data.db')
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM lattes_data WHERE name_link_link = ?", (name_link['link'],))
    existing_data = cursor.fetchall()
    
    lattes_data_array = []
    
    if existing_data:
        # Filtrar dados existentes para evitar duplicatas
        lattes_data_array = [elem for elem in existing_data if elem['name_link']['link'] != name_link['link']]
    
    # Adicionar dados de Lattes atuais ao array
    lattes_data_array.append({'name_link': name_link, 'pub_info': pub_info})
    
    # Salvar array de dados de Lattes no armazenamento local (SQLite)
    cursor.execute("INSERT INTO lattes_data (name_link, pub_info) VALUES (?, ?)", (json.dumps(name_link), json.dumps(pub_info)))
    conn.commit()
    conn.close()
    
    print("Informações de nome, link e publicação de Lattes salvas!")

In [ ]:
def annotate_published_articles(soup, qualis_data):
    # Localiza elementos contendo os artigos publicados
    articles_elements = soup.find_all('div', class_='published-article')
    
    # Inicializa lista para conter dados dos artigos publicados
    annotated_articles = []
    
    # Itera sobre cada elemento e anotando as informações necessárias
    for article in articles_elements:
        title = article.find('div', class_='article-title').text
        issn = article.find('div', class_='article-issn').text
        qualis = qualis_data.get(issn, 'N/A')  # Buscando o Qualis correspondente
        
        # Adiciona ao conjunto de artigos anotados
        annotated_articles.append({
            'title': title,
            'issn': issn,
            'qualis': qualis
        })
    
    # Persiste em SQLite
    conn = sqlite3.connect("lattes_data.db")
    cursor = conn.cursor()
    
    # Cria tabela de artigos publicados, se não existir
    cursor.execute("""
    CREATE TABLE IF NOT EXISTS published_articles (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        title TEXT,
        issn TEXT,
        qualis TEXT
    );
    """)
    
    # Insere artigos anotados na tabela
    for article in annotated_articles:
        cursor.execute("INSERT INTO published_articles (title, issn, qualis) VALUES (?, ?, ?)",
                       (article['title'], article['issn'], article['qualis']))
    
    # Commit e fecha a conexão
    conn.commit()
    conn.close()
    
async def annotate_lattes_page(driver, images_urls, qualis_data, qualis_data_cache, data_source_info):
    print("Procurando por publicações de periódicos...")
    
    # Inicializa BeautifulSoup para analisar o conteúdo HTML da página
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    
    # Extrai e anota informações dos artigos publicados
    qualis_info = await annotate_published_articles(driver, images_urls, qualis_data, qualis_data_cache, data_source_info, soup)
    
    return qualis_info

In [ ]:
def inject_annotation_message(images_urls, visualization_url, recent_updates_url, pub_count):
    annot_header_html = ''
    annot_buttons_html = ''
    pub_count_string = ''
    
    if pub_count > 0:
        s_char = 's' if pub_count > 1 else ''
        pub_count_string = f'anotou o Qualis de {pub_count} artigo{s_char} em periódico{s_char} neste CV.'
        annot_buttons_html = render_template_string("""
            <a href="#artigos-completos">
                <button> Ver anotações </button>
            </a>
        """)
    else:
        pub_count_string = 'não anotou nenhum artigo em periódico neste CV.'

    annot_header_html = render_template_string("""
        <a href="{{visualization_url}}" target="_blank" id="qlattes-logo">
            <img src="{{images_urls['qlattesLogoURL']}}" width="70">
        </a>{{pub_count_string}}
        </br>
    """, visualization_url=visualization_url, images_urls=images_urls, pub_count_string=pub_count_string)

    # Aqui a informação seria armazenada em um banco de dados em vez de ser injetada em um elemento HTML
    store_annotation_in_db(annot_header_html + annot_buttons_html)

In [ ]:
def consolidate_qualis_data(qualis_info):
    pub_data = []
    pub_data_year = []
    curr_year = 0
    
    for i in range(len(qualis_info)):
        if curr_year != qualis_info[i]['year']:
            if curr_year > 0:
                pub_data.append({'year': curr_year, 'pubList': pub_data_year})
                pub_data_year = []
            
            curr_year = qualis_info[i]['year']
        
        pub_data_item = {
            'issn': qualis_info[i]['issn'],
            'title': qualis_info[i]['title'],
            'pubName': qualis_info[i]['pubName'],
            'qualis': qualis_info[i]['qualisLabels']['qualis'],
            'baseYear': qualis_info[i]['qualisLabels']['baseYear'],
            'jcr': qualis_info[i]['jcrData']['jcr'] if 'jcrData' in qualis_info[i] and qualis_info[i]['jcrData'] else 0,
            'jcrYear': qualis_info[i]['jcrData']['baseYear'] if 'jcrData' in qualis_info[i] and qualis_info[i]['jcrData'] else ''
        }
        
        pub_data_year.append(pub_data_item)
        
    if len(qualis_info) > len(pub_data):
        pub_data.append({'year': curr_year, 'pubList': pub_data_year})
    
    return pub_data

In [ ]:
def create_text_link_html(text, tooltip, target_url):
    return render_template_string("""
        <a href="{{target_url}}" target="_blank" title="{{tooltip}}">{{text}}</a>
    """, text=text, tooltip=tooltip, target_url=target_url)

def create_icon_link_html(icon_url, icon_style, tooltip, target_url):
    return render_template_string("""
        <a href="{{target_url}}" target="_blank" title="{{tooltip}}">
            <img src="{{icon_url}}" style="{{icon_style}}">
        </a>
    """, icon_url=icon_url, icon_style=icon_style, tooltip=tooltip, target_url=target_url)

In [ ]:
def calculate_qualis_from_percentil(percentil):
    if not percentil:
        return 'N'

    qualis_class_list     = ['A1', 'A2', 'A3', 'A4', 'B1', 'B2', 'B3', 'B4']
    qualis_threshold_list = [87.5, 75, 62.5, 50, 37.5, 25, 12.5, 0]

    for i in range(len(qualis_threshold_list)):
        if percentil >= qualis_threshold_list[i]:
            return qualis_class_list[i]

    return 'N'


def get_alternative_issn(issn, capes_alt_data, scopus_data):
    # Pesquisar ISSN nos dados complementares da CAPES
    match = next((elem for elem in capes_alt_data if elem['issn'] == issn or elem['alt_issn'] == issn), None)
    
    if match:
        if 'alt_issn' in match and match['alt_issn'] != issn:
            return match['alt_issn']
        elif 'issn' in match and match['issn'] != issn:
            return match['issn']
        else:
            return ''
    else:
        # Pesquisar ISSN nos dados do Scopus
        match = next((elem for elem in scopus_data if elem['issn'] == issn or elem['e-issn'] == issn), None)
        
        if match:
            if 'e-issn' in match and len(match['e-issn']) > 0 and match['e-issn'] != issn:
                return match['e-issn']
            elif 'issn' in match and len(match['issn']) > 0 and match['issn'] != issn:
                return match['issn']
            else:
                return ''
        else:
            return ''

In [ ]:
def get_qualis_from_capes_data(issn, alt_issn, capes_data, scopus_data, data_source_info):
    qualis_labels = {
        'pubName': '',
        'qualis': 'N',
        'percentil': '',
        'linkScopus': '',
        'adjusted': '',
        'source': '',
        'baseYear': ''
    }

    # Procurar pelo ISSN na base de dados da CAPES
    match = next((elem for elem in capes_data if elem['issn'] == issn), None)
    
    if match:
        qualis_labels['source'] = 'capes'
    elif alt_issn != '':
        match = next((elem for elem in capes_data if elem['issn'] == alt_issn), None)
        
        if match:
            qualis_labels['source'] = 'capes_alt'
            
    if match:
        qualis_labels['qualis'] = match['qualis']
        qualis_labels['pubName'] = match['title'].upper()
        qualis_labels['baseYear'] = data_source_info[qualis_labels['source']]['baseYear']

        qualis_labels_scopus = get_qualis_from_scopus_data(issn, alt_issn, scopus_data, data_source_info)
        
        if qualis_labels_scopus['qualis'] != 'N':
            qualis_labels['linkScopus'] = qualis_labels_scopus['linkScopus']

    return qualis_labels


def get_qualis_from_pucrs_data(issn, alt_issn, pub_name, pucrs_data, data_source_info):
    qualis_labels = {
        'pubName': '',
        'qualis': 'N',
        'percentil': '',
        'linkScopus': '',
        'adjusted': '',
        'source': '',
        'baseYear': ''
    }

    labels_map = {
        'pubName': 'periodico',
        'qualis': 'Qualis_Final',
        'percentil': 'percentil',
        'linkScopus': 'link_scopus',
        'adjusted': 'Ajuste_SBC'
    }
    
    match = next((elem for elem in pucrs_data if elem['issn'] == issn or (alt_issn and elem['issn'] == alt_issn)), None)
    
    if match:
        for key in labels_map.keys():
            if labels_map[key] in match and match[labels_map[key]] != 'nulo':
                qualis_labels[key] = match[labels_map[key]]
                
        qualis_labels['source'] = 'pucrs'
        qualis_labels['baseYear'] = data_source_info['pucrs']['baseYear']

    return qualis_labels


def get_qualis_from_scopus_data(issn, alt_issn, scopus_data, data_source_info):
    qualis_labels = {
        'pubName': '',
        'qualis': 'N',
        'percentil': '',
        'linkScopus': '',
        'adjusted': '',
        'source': '',
        'baseYear': ''
    }

    # Procura pelo ISSN nos dados da Scopus
    match = next((elem for elem in scopus_data if elem['issn'] == issn or elem['e-issn'] == issn), None)
    
    if not match and alt_issn != '':
        match = next((elem for elem in scopus_data if elem['issn'] == alt_issn or elem['e-issn'] == alt_issn), None)
        
    if match:
        qualis_labels['qualis'] = calculate_qualis_from_percentil(match['percentil'])
        qualis_labels['pubName'] = match['title'].upper()
        qualis_labels['percentil'] = match['percentil']
        qualis_labels['linkScopus'] = match['source-id-url']
        qualis_labels['source'] = 'scopus'
        qualis_labels['baseYear'] = data_source_info['scopus']['baseYear']
    
    return qualis_labels


async def get_qualis(issn, pub_name, qualis_data, qualis_data_cache, data_source_info):
    qualis_labels = {
        'pubName': '',
        'qualis': 'N',
        'percentil': '',
        'linkScopus': '',
        'adjusted': '',
        'source': '',
        'baseYear': ''
    }

    if not issn:
        return qualis_labels

    alt_issn = ''

    # Verificar se o ISSN já está em cache
    if issn in qualis_data_cache:
        return qualis_data_cache[issn]
    else:
        # Verificar se um ISSN alternativo existe e está em cache
        alt_issn = await get_alternative_issn(issn, qualis_data['capes_alt'], qualis_data['scopus'])
        if alt_issn and alt_issn in qualis_data_cache:
            return qualis_data_cache[alt_issn]

    # Procurar pelo ISSN nos dados CAPES
    qualis_labels = await get_qualis_from_capes_data(
        issn, alt_issn, qualis_data['capes'], qualis_data['scopus'], data_source_info
    )

    # Se não encontrado
    if qualis_labels['qualis'] == 'N':
        # Procurar pelo ISSN nos dados PUC-RS
        qualis_labels = await get_qualis_from_pucrs_data(
            issn, alt_issn, pub_name, qualis_data['pucrs'], data_source_info
        )

        # Se ainda não encontrado
        if qualis_labels['qualis'] == 'N':
            # Procurar pelo ISSN nos dados Scopus
            qualis_labels = await get_qualis_from_scopus_data(
                issn, alt_issn, qualis_data['scopus'], data_source_info
            )

    # Adicionar rótulos ao cache Qualis
    qualis_data_cache[issn] = qualis_labels

    if alt_issn:
        qualis_data_cache[alt_issn] = qualis_labels

    return qualis_labels

In [ ]:
def persist_qualis_annotation(pub_info, images_URLs, data_source_info):
    annotation_dict = {}
    
    # Create QLattes icon element
    qlattes_img_elem = BeautifulSoup('<img>', 'html.parser')
    qlattes_img_elem['src'] = images_URLs['qlattesIconURL']
    qlattes_img_elem['style'] = 'margin-bottom:-4px'
    
    annotation_dict['qlattes_img_elem'] = str(qlattes_img_elem)
    
    # Create Qualis labels annotations
    issn_label = f', ISSN {pub_info["issn"]}' if pub_info.get('issn') else ''
    
    if pub_info['qualisLabels']['qualis'] == 'N':
        qualis_annot = f' Não classificado{issn_label}'
    else:
        qualis_annot = f' {pub_info["qualisLabels"]["qualis"]}{issn_label}'
        
        # add Data source and base year
        source = pub_info['qualisLabels']['source']
        source_info = data_source_info[source]
        
        data_source_label = f'{source_info["label"]} ({source_info["baseYear"]})'
        qualis_annot += f', fonte {data_source_label}'
    
    annotation_dict['qualis_annot'] = qualis_annot
    
    # add icon with link to Google Scholar
    base_url = 'https://scholar.google.com/scholar?q='
    title_param = f'intitle%3A%22{pub_info["title"].replace(" ", "+")}%22'
    link_scholar = f'{base_url}{title_param}'
    
    annotation_dict['link_scholar'] = link_scholar
    
    # add icon with link to Scopus (if available)
    if pub_info['qualisLabels'].get('linkScopus'):
        annotation_dict['link_scopus'] = pub_info['qualisLabels']['linkScopus']
    
    return annotation_dict

def persist_annotation_div():
    alert_div = BeautifulSoup('<div>', 'html.parser')
    
    alert_div['class'] = 'main-content max-width min-width'
    alert_div['id'] = 'annot-div'
    
    print('Alert div persisted!')
    
    return str(alert_div)


def persist_qualis_annotation(elem, pubInfo, imagesURLs, dataSourceInfo):
    annot_dict = {}
    
    issnLabel = f", ISSN {pubInfo['issn']}" if pubInfo['issn'] else pubInfo['issn']
    if pubInfo['qualisLabels']['qualis'] == 'N':
        qualisAnnot = f"Não classificado{issnLabel}"
    else:
        qualisAnnot = f"{pubInfo['qualisLabels']['qualis']}{issnLabel}"
        
        dataSourceLabel = dataSourceInfo[pubInfo['qualisLabels']['source']]['label']
        baseYear = dataSourceInfo[pubInfo['qualisLabels']['source']]['baseYear']
        dataSourceLabel += f" ({baseYear})"
        qualisAnnot += f", fonte {dataSourceLabel}"
        
    annot_dict['qualisAnnot'] = qualisAnnot
    
    titleParam = f"intitle:\"{pubInfo['title']}\""
    linkScholar = f"https://scholar.google.com/scholar?q={titleParam}"
    annot_dict['linkScholar'] = linkScholar
    
    if pubInfo['qualisLabels'].get('linkScopus'):
        annot_dict['linkScopus'] = pubInfo['qualisLabels']['linkScopus']
    
    elem['annotation'] = annot_dict

def persist_annotation_div(imagesURLs, visualizationURL):
    alert_div_dict = {}
    alert_div_dict['id'] = "annot-div"
    return alert_div_dict

def persist_annotation_message(imagesURLs, visualizationURL, recentUpdatesURL, pubCount):
    annot_dict = {}
    if pubCount > 0:
        sChar = 's' if pubCount > 1 else ''
        pubCountString = f"anotou o Qualis de {pubCount} artigo{sChar} em periódico{sChar}  neste CV."
    else:
        pubCountString = "não anotou nenhum artigo em periódico neste CV."
        
    annot_dict['pubCountString'] = pubCountString
    annot_dict['visualizationURL'] = visualizationURL
    annot_dict['recentUpdatesURL'] = recentUpdatesURL
    return annot_dict

def set_attributes(elem, attrs):
    for key, value in attrs.items():
        elem[key] = value

def consolidate_qualis_data(qualisInfo):
    pubData = []
    pubDataYear = []
    currYear = 0
    
    for qInfo in qualisInfo:
        if currYear != qInfo['year']:
            if currYear > 0:
                pubData.append({'year': currYear, 'pubList': pubDataYear})
                pubDataYear = []
            currYear = qInfo['year']
        
        pubDataItem = {
            'issn': qInfo['issn'],
            'title': qInfo['title'],
            'pubName': qInfo['pubName'],
            'qualis': qInfo['qualisLabels']['qualis'],
            'baseYear': qInfo['qualisLabels']['baseYear'],
            'jcr': qInfo['jcrData']['jcr'] if 'jcr' in qInfo['jcrData'] else 0,
            'jcrYear': qInfo['jcrData']['baseYear'] if 'baseYear' in qInfo['jcrData'] else ''
        }
        pubDataYear.append(pubDataItem)
    
    if len(qualisInfo) > len(pubData):
        pubData.append({'year': currYear, 'pubList': pubDataYear})
        
    return pubData

# Suponhamos que 'html_content' seja o conteúdo HTML em que as anotações serão inseridas.
# html_content = ...

# Criamos um objeto BeautifulSoup
# soup = BeautifulSoup(html_content, 'html.parser')

# Aqui você pode utilizar os métodos acima para persistir as informações.
# Exemplo:
# elem = {}
# persist_qualis_annotation(elem, pubInfo, imagesURLs, dataSourceInfo)


In [ ]:
# HTML annotation and CV ID
annot_html = "<a href='some_url'>Annotated Data</a>"
cv_id = "cv_123"

# Store annotation
store_annotation_in_db(uri, user, password, annot_html, cv_id)

# Execução na nova abordagem

# Variações das funções

In [ ]:
# async def process_lattes_page_v1(name_link, images_urls, recent_updates_url, qualis_data, data_source_info): 
#     print(qualis_data)
    
#     # Inicializando o WebDriver
#     driver_service = Service('path/to/chromedriver')
#     driver = webdriver.Chrome(service=driver_service)
#     driver.get(name_link['link'])
    
#     # URL para visualização (pode ser determinado conforme a necessidade)
#     visualization_url = 'path/to/visualization.html'
    
#     # Limpar cache de dados de Qualis
#     qualis_data_cache = {}
    
#     # Annotate Lattes page com informações de Qualis
#     lattes_info = await annotate_lattes_page(driver, images_urls, qualis_data, qualis_data_cache, data_source_info)
    
#     if lattes_info:
#         await inject_annotation_message(images_urls, visualization_url, recent_updates_url, len(lattes_info))
    
#     # Consolidar dados de publicação a partir das informações de Lattes
#     pub_info = consolidate_qualis_data(lattes_info)
#     print(f"pubInfo: {pub_info}")
    
#     # Tentativa de ler dados de Lattes do armazenamento local (neste caso, SQLite)
#     conn = sqlite3.connect('lattes_data.db')
#     cursor = conn.cursor()
#     cursor.execute("SELECT * FROM lattes_data WHERE name_link_link = ?", (name_link['link'],))
#     existing_data = cursor.fetchall()
    
#     lattes_data_array = []
    
#     if existing_data:
#         # Filtrar dados existentes para evitar duplicatas
#         lattes_data_array = [elem for elem in existing_data if elem['name_link']['link'] != name_link['link']]
    
#     # Adicionar dados de Lattes atuais ao array
#     lattes_data_array.append({'name_link': name_link, 'pub_info': pub_info})
    
#     # Salvar array de dados de Lattes no armazenamento local (SQLite)
#     cursor.execute("INSERT INTO lattes_data (name_link, pub_info) VALUES (?, ?)", (json.dumps(name_link), json.dumps(pub_info)))
#     conn.commit()
#     conn.close()
    
#     print("Informações de nome, link e publicação de Lattes salvas!")

In [ ]:
# async def annotate_published_articles_v2(driver, images_urls, qualis_data, qualis_data_cache, data_source_info, soup):
#     # Localizar o primeiro elemento de artigo publicado
#     start_elem = soup.find("div", id="artigos-completos")

#     # Retornar uma lista vazia se não houver nenhum artigo publicado no CV
#     if start_elem is None:
#         return []

#     qualis_info = []

#     # Encontrar todos os artigos publicados
#     pub_elems = start_elem.find_all("div", class_="artigo-completo")

#     for pub_elem in pub_elems:
#         qualis_pub_info = {
#             'year': None,
#             'issn': '',
#             'title': '',
#             'pubName': '',
#             'qualisLabels': '',
#             'jcrData': {}
#         }
#         # Obter o ano de publicação
#         year_span = pub_elem.find("span", class_="informacao-artigo", attrs={"data-tipo-ordenacao": "ano"})
#         if year_span:
#             qualis_pub_info['year'] = int(year_span.text)
        
#         # Obter dados de publicação
#         pub_elem_data = pub_elem.find("div", attrs={"cvuri": True})

#         if pub_elem_data:
#             # Obter informações do periódico
#             pub_info_string = pub_elem_data['cvuri']
#             # Para fins de simplicidade, omitimos a função escapeHtml já que não é relevante para o BeautifulSoup

#             # Obter ISSN, título e nome do periódico
#             # Detalhes de implementação podem variar, pois o exemplo original usa JavaScript para manipular atributos DOM
#             issn = ''  # Implemente a lógica para extrair o ISSN
#             title = ''  # Implemente a lógica para extrair o título
#             pub_name = ''  # Implemente a lógica para extrair o nome do periódico
            
#             qualis_pub_info['issn'] = issn
#             qualis_pub_info['title'] = title
#             qualis_pub_info['pubName'] = pub_name.upper()

#             # Obter classificação Qualis do periódico
#             qualis_labels = await get_qualis(issn, pub_name, qualis_data, qualis_data_cache, data_source_info)
#             qualis_pub_info['qualisLabels'] = qualis_labels

#             # Obter dados JCR (omitido neste exemplo; pode ser implementado conforme a necessidade)
            
#             # Adicionar informações ao vetor qualis_info
#             qualis_info.append(qualis_pub_info)
            
#     return qualis_info

# Funções

In [ ]:
def definir_sistema(pastaraiz):
    import os
    import sys
    sistema_operacional =sys.platform

    try:
        if 'linux' in sistema_operacional:
            print('Sistema operacional Linux')
            try:
                drive   = '/home/'
                usuario = 'mak/'
                os.listdir(drive+usuario)
            except:
                drive   = '/home/'
                usuario = 'marcos/'
        elif 'win32' in sistema_operacional:
            print('Sistema operacional Windows')
            drive   = 'C'
            print(f'Drive em uso {drive.upper()}')
            # drive = 'E'
            # drive = input('Indique qual a letra da unidade onde deseja armazenar os arquivos (Ex.: C, E...)')
            usuario = 'Users/marco/'
            if os.path.isdir(drive+':/'+usuario) is False:
                usuario = 'Users/marcos.aires/'
        else:
            print('SO não reconhecido')

    except Exception as e:
        print('Erro ao preparar as pastas locais!')
        print(e)

    caminho = drive+':/'+usuario+pastaraiz
    
    print(f'Pasta armazenagem local {caminho}\n')

    return caminho, drive, usuario

def preparar_pastas(caminho):
    # caminho, drive, usuario = definir_sistema(pastaraiz)
    # # caminho = drive+':/'+usuario+pastaraiz
    # caminho = drive+':/'+pastaraiz
    if os.path.isdir(caminho) is False:
        os.mkdir(caminho)
        if os.path.isdir(caminho+'/xml_zip'):
            print ('Pasta para os arquivo xml já existe!')
        else:
            os.mkdir(caminho+'/xml_zip')
            print ('Pasta para arquivo xml criada com sucesso!')
        if os.path.isdir(caminho+'/csv'):
            print ('Pasta para os arquivo CSV já existe!')
        else:
            os.mkdir(caminho+'/csv')
            print ('Pasta para arquivo CSV criada com sucesso!')
        if os.path.isdir(caminho+'/json'):
            print ('Pasta para os arquivo JSON já existe!')
        else:
            os.mkdir(caminho+'/json')
            print ('Pasta para JSON criada com sucesso!')
        if os.path.isdir(caminho+'/fig'):
            print ('Pasta para figuras já existe!')
        else:
            os.mkdir(caminho+'/fig')
            print ('Pasta para JSON criada com sucesso!')
    else:
        if os.path.isdir(caminho+'/xml_zip'):
            print ('Pasta para os xml já existe!')
        else:
            os.mkdir(caminho+'/xml_zip')
            print ('Pasta para xml criada com sucesso!')
        if os.path.isdir(caminho+'/csv'):
            print ('Pasta para os CSV já existe!')
        else:
            os.mkdir(caminho+'/csv')
            print ('Pasta para CSV criada com sucesso!')
        if os.path.isdir(caminho+'/json'):
            print ('Pasta para os JSON já existe!')
        else:
            os.mkdir(caminho+'/json')
            print ('Pasta para JSON criada com sucesso!')
        if os.path.isdir(caminho+'/fig'):
            print ('Pasta para figuras já existe!')
        else:
            os.mkdir(caminho+'/fig')
            print ('Pasta para figuras criada com sucesso!')
        if os.path.isdir(caminho+'/output'):
            print ('Pasta para saídas já existe!')
        else:
            os.mkdir(caminho+'/output')
            print ('Pasta para saídas criada com sucesso!')            

    pathzip  = caminho+'xml_zip/'
    pathcsv  = caminho+'csv/'
    pathjson = caminho+'json/'
    pathfig  = caminho+'fig/'
    pathaux  = caminho
    pathout  = caminho+'output/'

    print('\nCaminho da pasta raiz', pathaux)
    print('Caminho arquivos  XML', pathzip)
    print('Caminho arquivos JSON', pathjson)
    print('Caminho arquivos  CSV', pathcsv)
    print('Caminho para  figuras', pathfig)
    print('Pasta arquivos saídas', pathout)
    
    return pathzip, pathcsv, pathjson, pathfig, pathaux, pathout

def strfdelta(tdelta, fmt='{H:02}h {M:02}m {S:02}s', inputtype='timedelta'):
    """Convert a datetime.timedelta object or a regular number to a custom-formatted string, 
    just like the stftime() method does for datetime.datetime objects.

    The fmt argument allows custom formatting to be specified.  Fields can 
    include seconds, minutes, hours, days, and weeks.  Each field is optional.

    Some examples:
        '{D:02}d {H:02}h {M:02}m {S:02}s' --> '05d 08h 04m 02s' (default)
        '{W}w {D}d {H}:{M:02}:{S:02}'     --> '4w 5d 8:04:02'
        '{D:2}d {H:2}:{M:02}:{S:02}'      --> ' 5d  8:04:02'
        '{H}h {S}s'                       --> '72h 800s'

    The inputtype argument allows tdelta to be a regular number instead of the  
    default, which is a datetime.timedelta object.  Valid inputtype strings: 
        's', 'seconds', 
        'm', 'minutes', 
        'h', 'hours', 
        'd', 'days', 
        'w', 'weeks'
    """

    # Convert tdelta to integer seconds.
    if inputtype == 'timedelta':
        remainder = int(tdelta.total_seconds())
    elif inputtype in ['s', 'seconds']:
        remainder = int(tdelta)
    elif inputtype in ['m', 'minutes']:
        remainder = int(tdelta)*60
    elif inputtype in ['h', 'hours']:
        remainder = int(tdelta)*3600
    elif inputtype in ['d', 'days']:
        remainder = int(tdelta)*86400
    elif inputtype in ['w', 'weeks']:
        remainder = int(tdelta)*604800

    f = Formatter()
    desired_fields = [field_tuple[1] for field_tuple in f.parse(fmt)]
    possible_fields = ('W', 'D', 'H', 'M', 'S')
    constants = {'W': 604800, 'D': 86400, 'H': 3600, 'M': 60, 'S': 1}
    values = {}
    
    for field in possible_fields:
        if field in desired_fields and field in constants:
            values[field], remainder = divmod(remainder, constants[field])
    
    return f.format(fmt, **values)

def tempo(start, end):
    t=end-start

    tempo = timedelta(
        weeks   = t//(3600*24*7),
        days    = t//(3600*24),
        seconds = t,
        minutes = t//(60),
        hours   = t//(3600),
        microseconds=t//1000000,
        )
    fmt='{H:2}:{M:02}:{S:02}'
    return strfdelta(tempo)

# https://sh-tsang.medium.com/tutorial-cuda-cudnn-anaconda-jupyter-pytorch-installation-in-windows-10-96b2a2f0ac57

def try_amb():
    ## Visualizar versões dos principais componentes
    import os
    import pip
    import sys
    import warnings
    warnings.filterwarnings("ignore", category=UserWarning)
    warnings.filterwarnings("ignore", category=DeprecationWarning)
    # !pip3 install shutup
    # import shutup; shutup.please()
    
    pyVer      = sys.version
    pipVer     = pip.__version__
    
    print('\nVERSÕES DAS PRINCIPAIS BIBLIOTECAS INSTALADAS NO ENVIROMENT')
    print('Interpretador em uso:', sys.executable)
    print('    Ambiente ativado:',os.environ['CONDA_DEFAULT_ENV'])
    print('     Python: '+pyVer, '\n        Pip:', pipVer,'\n'
         )

    !nvcc -V

def try_gpu():
    print('\nVERSÕES DO PYTORCH E GPU DISPONÍVEIS')
    try:
        import torch
        print('    PyTorch:',torch.__version__)
        device = 'cuda' if torch.cuda.is_available() else 'cpu'
        print('Dispositivo:',device)
        print('Disponível :',device,torch.cuda.is_available(),' | Inicializado:',torch.cuda.is_initialized(),'| Capacidade:',torch.cuda.get_device_capability(device=None))
        print('Nome GPU   :',torch.cuda.get_device_name(0),'         | Quantidade:',torch.cuda.device_count(),'\n')
    except Exception as e:
        print('Erro ao configurar a GPU:',e,'\n')

def try_folders(drives,pastas,pastasraiz):
    import os
    for drive in drives:
        for i in pastas:
            for j in pastasraiz:
                try:
                    caminho_testado = drive+i+j
                    if os.path.isfile(caminho_testado+'/chromedriver/chromedriver.exe'):
                        print(f"Listando arqivos em: {caminho_testado}")
                        print(os.listdir(caminho_testado))
                        caminho = caminho_testado+'/'
                except:
                    caminho=''
                    print('Não foi possível encontrar uma pasta de trabalho')
    return caminho

def try_browser(raiz):
    print('\nVERSÕES DO BROWSER E DO CHROMEDRIVER INSTALADAS')
    from selenium import webdriver
    from selenium import webdriver
    from selenium.webdriver.chrome.service import Service

    try:
        # Caminho para o seu chromedriver
        driver_path=raiz+'chromedriver/chromedriver.exe'
        print(driver_path)
        service = Service(driver_path)
        driver = webdriver.Chrome(service=service)
        str1 = driver.capabilities['browserVersion']
        str2 = driver.capabilities['chrome']['chromedriverVersion'].split(' ')[0]
        print(f'     Versão do browser: {str1}')
        print(f'Versão do chromedriver: {str2}')
        driver.quit()

        if str1[0:3] != str2[0:3]: 
            print("Versões incompatíveis, atualizar chromedriver!")
            print('  Baixar versão atualizada do Chromedriver em:')
            print('  https://googlechromelabs.github.io/chrome-for-testing/#stable')
            print('     Ex. Versão 116 PARA WINDOWS:')
            print('	    https://edgedl.me.gvt1.com/edgedl/chrome/chrome-for-testing/116.0.5845.96/win64/chromedriver-win64.zip')
    except Exception as e:
        print(e)

def try_chromedriver(caminho):
    try:
        import os
        os.listdir(caminho)
    except Exception as e:
        raiz=caminho

    finally:
        print(raiz)
    return raiz

def conectar_busca():    
    print(f'Conectando com o servidor do CNPq...')
    import time
    # print(f'Iniciada extração de {len(lista_nomes)} currículos')
    t0=time.time()
    
    ## https://www.selenium.dev/documentation/pt-br/webdriver/browser_manipulation/
    options   = Options()
    # options.add_argument("--headless")
    browser   = webdriver.Chrome(options=options)
    url_buscaespecialista = 'http://buscatextual.cnpq.br/buscatextual/busca.do?buscarDoutores=true&buscarDemais=true&textoBusca='
    browser.get(url_buscaespecialista) # acessa a url de busca do CNPQ   
    
    browser.set_window_position(-20, -10)
    browser.set_window_size(170, 1896)
    browser.mouse = webdriver.ActionChains(browser)
    # url        = browser.command_executor._url #"http://127.0.0.1:60622/hub"
    # session_id = browser.session_id            #'4e167f26-dc1d-4f51-a207-f761eaf73c31'
         
    t1 = time.time()
    tcon = tempo(t0,t1)
    print(f'{tcon} para conectar ao servidor do CNPq')
    # print('Conectado com sucesso em:', url, session_id)   
    time.sleep(0.00001)
    
    # return browser, url, session_id
    return browser

def retry(func,
          expected_ex_type=Exception,
          limit=0,
          wait_ms=100,
          wait_increase_ratio=2,
          logger=None,
          on_exhaust="throw"
          ):
    """
    Retry a function invocation until no exception occurs
    :param func: function to invoke
    :param expected_ex_type: retry only if exception is subclass of this type
    :param limit: maximum number of invocation attempts, 0 for unlimited attempts.
    :param wait_ms: initial wait time after each attempt in milliseconds.
    :param wait_increase_ratio: increase wait period by multiplying this value after each attempt.
    :param logger: if not None, retry attempts will be logged to this logging.logger
    :param on_exhaust: return value when retry attempts exhausted. Default is "throw" which will rethrow the exception
                 of the last attempt.
    :return: result of first successful invocation
    :raises: last invocation exception if attempts exhausted or exception is not an instance of ex_type
    """

    attempt = 1
    while True:
        try:
            return func()
        except Exception as ex:
            if not isinstance(ex, expected_ex_type):
                raise ex

            if logger:
                logger.error("Failed execution attempt #%d", attempt, exc_info=ex)

            # check if attempts exhausted
            if 0 < limit <= attempt:
                if logger:
                    logger.warning("Attempt limit (%d) reached", limit)
                if on_exhaust == "throw":
                    raise ex
                return on_exhaust

            # prepare for next attempt
            attempt += 1
            if logger:
                logger.info("Waiting %d ms before attempt #%d", wait_ms, attempt)
            time.sleep(wait_ms / 1000)
            wait_ms *= wait_increase_ratio

def paginar(browser):
    try:
        css_paginacao="div.paginacao:nth-child(2)"  #seletorcss="div.paginacao:nth-child(4) > a:nth-child(2)"
        WebDriverWait(browser, delay).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, css_paginacao)))
        paginacao = browser.find_element(By.CSS_SELECTOR, css_paginacao)
        paginas=paginacao.text.split(' ')
        remover=['','anterior','...']
        numpaginas = [x for x in paginas if x not in remover]
        # print('NumPáginas interno:',numpaginas)
    except Exception as e:
        print('Erro ao utilizar função paginar():', e)
    return numpaginas

def achar_busca(browser, delay):
    '''
    Função para clicar no botão Buscar Currículo
    '''
    delay=10
    try:
        limite=5
        xpath_nome = "/html/body/form/div/div[4]/div/div/div/div[3]/div/div[3]/ol/li/b"
        retry(WebDriverWait(browser, delay).until(
            EC.element_to_be_clickable((By.XPATH, xpath_nome))),
            #    expected_ex_type=ZeroDivisionError, 
               wait_ms=20,
               limit=limite, 
            #    logger=logger, 
               on_exhaust=(f'Problema ao acessar ao servidor do CNpQ função definir_filtros(). {limite} tentativas sem sucesso.'))   

        link_nome  = browser.find_element(By.XPATH, xpath_nome)
        # ActionChains(browser).move_to_element(link_nome).perform()
        
        # Avaliar 'Stale file handle'
        if link_nome.text == None:
            xpath_nome = '/html/body/form/div/div[4]/div/div/div/div[3]/div/div[3]/ol/li'
            
            print('Sem resposta do servidor, tentando novamente...')
            retry(WebDriverWait(browser, delay).until(
                EC.element_to_be_clickable((By.XPATH, xpath_nome))),
            #    expected_ex_type=ZeroDivisionError, 
               wait_ms=200,
               limit=limite, 
            #    logger=logger, 
               on_exhaust=(f'Problema ao acessar ao servidor do CNpQ função definir_filtros(). {limite} tentativas sem sucesso.'))   

        return link_nome

    except TimeoutException as t:
        print(f'Erro de conexão durante achar_busca(). Reiniciando a função dentro de 2 segundos...')
        time.sleep(2)

def preencher_busca(browser, delay, NOME):
    '''
    Função para passar o nome para campo de busca
    '''
    try:
        nome = lambda: browser.find_element(By.CSS_SELECTOR, ("#textoBusca"))
        nome().send_keys(Keys.CONTROL + "a")
        nome().send_keys(NOME)
        # browser.find_element(By.CSS_SELECTOR, ("#textoBusca")).send_keys(Keys.ENTER)
        seletorcss='div.layout-cell-12:nth-child(8) > div:nth-child(1) > div:nth-child(1) > a:nth-child(1)'
        # seletorcss="#textoBusca"  
        
        time.sleep(0.5)
        WebDriverWait(browser, delay).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, seletorcss))).click()
        seletorcss="#botaoBuscaFiltros"  
        
        WebDriverWait(browser, delay).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, seletorcss)))
        
    except Exception as e:
        traceback_str = ''.join(traceback.format_tb(e.__traceback__))          
        print('Erro ao preencher nome no campo de busca, pausando por 1 segundo')
        print(e,traceback_str)
        # print(e)
        # time.sleep(1.5)

def definir_filtros(browser, delay, mestres=True, assunto=False):
    '''
    Clica nos check-boxes para definir os filtros de buscas
    Para buscar por Assuntos usar parâmetro True, caso omitido fará busca por Nome por default
    '''
    from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException, TimeoutException
    from selenium.webdriver.support import expected_conditions as EC
    from selenium.webdriver.support.ui import WebDriverWait
    from selenium.webdriver.common.by import By
    import traceback   
    
    ## Aguardar carregar e clicar em checkbox de Assunto
    try:
        if mestres == True:
            limite=2
            ## Aguardar opção dropdown ser carregada e clicar em sua checkbox
            css_buscar_demais = '#buscarDemais'
            checkbox_buscar_demais = browser.find_element(By.CSS_SELECTOR, css_buscar_demais)
            
            retry(WebDriverWait(browser, delay).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, css_buscar_demais))),
                   wait_ms=150,
                   limit=limite, 
                   on_exhaust=(f'Problema clicar em {checkbox_buscar_demais}, {limite} tentativas sem sucesso.'))
            
            time.sleep(1)
            checkbox_buscar_demais.click()
            print(f'Clique efetuado em {checkbox_buscar_demais}')

        # css_estrangeiros  = '#buscarEstrangeiros'

        if assunto == True:
            ## Implementar número de retentativas para casos de conexão muito instável
            limite=5
            retry(WebDriverWait(browser, delay).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, "#buscaAssunto"))).click(),
                #    expected_ex_type=ZeroDivisionError, 
                   wait_ms=200,
                   limit=limite, 
                #    logger=logger, 
                   on_exhaust=(f'Problema ao acessar ao servidor do CNpQ função definir_filtros(). {limite} tentativas sem sucesso.'))   
        
            ## Aguardar opção Atuação Profissional ser carregada e clicar em sua checkbox
            xpath_atuacaoprofissional = ".//*[contains(text(), 'Atuação profissional')]"
            WebDriverWait(browser, delay).until(
                EC.element_to_be_clickable((By.XPATH, xpath_atuacaoprofissional))).click()

            ## Aguardar opção Ciências da Saúde ser carregada e clicar em sua checkbox
            xpath_cienciassaude = ".//*[contains(text(), 'Ciências da Saúde')]"
            WebDriverWait(browser, delay).until(
                EC.element_to_be_clickable((By.XPATH, xpath_cienciassaude))).click()
            #browser.find_element_by_xpath(xpath_cienciassaude).click()        

            ## Aguardar opção Enfermagem ser carregada e clicar em sua checkbox
            xpath_enfermagem    = ".//*[contains(text(), 'Enfermagem')]"
            WebDriverWait(browser, delay).until(
                EC.element_to_be_clickable((By.XPATH, xpath_enfermagem))).click()
            #browser.find_element_by_xpath(xpath_enfermagem).click()
            aplicar_link  = browser.find_element(By.LINK_TEXT, 'Aplicar')
            aplicar_link.click()
   
    except Exception as e:
        print(f'Erro na função definir_filtros()')
        print(e)
        # traceback_str = ''.join(traceback.format_tb(e.__traceback__))
        # print(e, traceback_str) 

def procurar_vinculos(NOME, instituicao, unidade, termo, browser, delay, limite):
    from selenium.common import exceptions
    from pyjarowinkler.distance import get_jaro_distance
    ignored_exceptions=(NoSuchElementException,StaleElementReferenceException,)
    ## Receber a quantidade de opções ao ler elementos de resultados
    duvidas   = []
    force_break_loop = False
    try:
        css_resultados = ".resultado"
        WebDriverWait(browser, delay).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, css_resultados)))
        resultados = browser.find_elements(By.CSS_SELECTOR, css_resultados)       
        ## Ler quantidade de resultados apresentados pela busca de nome
        try:
            css_qteresultados = ".tit_form > b:nth-child(1)"
            WebDriverWait(browser, delay).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, css_qteresultados)))                       
            soup = BeautifulSoup(browser.page_source, 'html.parser')
            div_element = soup.find('div', {'class': 'tit_form'})
            match = re.search(r'<b>(\d+)</b>', str(div_element))
            if match:
                qte_resultados = int(match.group(1))
                print(f'{qte_resultados} resultados para {NOME}')
            else:
                raise Exception
        except Exception as e:
            print('Erro ao ler a quantidade de resultados:')
            print(e)
            return np.NaN, NOME, np.NaN, e, browser
        
        ## Escolher função a partir da quantidade de resultados da lista apresentada na busca
        ## Ao achar clica no elemento elm_vinculo com link do nome para abrir o currículo
        numpaginas = paginar(browser)
        if numpaginas == [] and qte_resultados==1:
            # capturar link para o primeiro nome resultado da busca
            ## TO-DO inserir a crítica para o 'Stale file handle'
            try:
                css_linknome = ".resultado > ol:nth-child(1) > li:nth-child(1) > b:nth-child(1) > a:nth-child(1)"
                WebDriverWait(browser, delay).until(EC.presence_of_element_located((By.CSS_SELECTOR, css_linknome)))            
                elm_vinculo  = browser.find_element(By.CSS_SELECTOR, css_linknome)
                nome_vinculo = elm_vinculo.text
            except Exception as e:
                print('Erro ao encontrar o primeiro resultado da lista de nomes:',e)
                return np.NaN, NOME, np.NaN, e, browser
            # print('Clicar no nome único:', nome_vinculo)
            try:
                retry(ActionChains(browser).click(elm_vinculo).perform(),
                       wait_ms=20,
                       limit=limite,
                       on_exhaust=(f'Problema ao clicar no link do nome. {limite} tentativas sem sucesso.'))   
            except Exception as e:
                print('Erro ao clicar no único nome encontrado anteriormente',e)
                return np.NaN, NOME, np.NaN, e, browser
        
        ## Quantidade de resultados até 10 currículos, acessados sem paginação
        else:
            print(f'{qte_resultados} currículos de homônimos em potencial...')
            numpaginas = paginar(browser)
            numpaginas.append('próximo')
            iteracoes=0
            ## iterar em cada página de resultados
            pagin = qte_resultados//10+1
            for i in range(pagin+1):
                # print(i,'/',pagin)
                iteracoes+=1
                try:
                    numpaginas = paginar(browser)
                    print(f'Iteração: {iteracoes}. Páginas sendo lidas: {numpaginas}')
                    css_resultados = ".resultado"
                    WebDriverWait(browser, delay).until(
                        EC.presence_of_element_located((By.CSS_SELECTOR, css_resultados)))
                    resultados = browser.find_elements(By.CSS_SELECTOR, css_resultados)
                except Exception as e:
                    print('Erro ao paginar:',e)
                ## iterar em cada resultado
                for n,i in enumerate(resultados):
                    linhas = i.text.split('\n\n')
                    # print(linhas)
                    if 'Stale file handle' in str(linhas):
                        return np.NaN, NOME, np.NaN, 'Stale file handle', browser
                    for m,linha in enumerate(linhas):
                        # print(f'\nOrdem da linha: {m+1}, de total de linhas {len(linhas)}')
                        # print('Conteúdo da linha:',linha.lower())
                        print(linha)
                        try:
                            if instituicao.lower() in linha.lower() or unidade.lower() in linha.lower() or termo.lower() in linha.lower():
                                # print('Vínculo encontrado!')
                                count=m
                                # print(' NOME:', NOME, type(NOME))
                                # test = linhas[count].split('\n')[0]
                                # print('TESTE:',test, type(test))
                                while get_jaro_distance(linhas[count].split('\n')[0], str(NOME)) < 0.75:
                                    count-=1
                                print('Identificado vínculo com o interesse de análise no resultado:', m+1)
                                nome_vinculo = linhas[count].strip()
                                print(f'    Achado: {nome_vinculo}')
                                try:
                                    css_vinculo = f".resultado > ol:nth-child(1) > li:nth-child({m+1}) > b:nth-child(1) > a:nth-child(1)"
                                    # print('\nCSS_SELECTOR usado:', css_vinculo)
                                    css_alvo = '.resultado > ol:nth-child(1) > li:nth-child(7) > b:nth-child(1) > a:nth-child(1)'
                                    WebDriverWait(browser, delay).until(EC.presence_of_element_located((By.CSS_SELECTOR, css_vinculo)))            
                                    elm_vinculo  = browser.find_element(By.CSS_SELECTOR, css_vinculo)
                                    nome_vinculo = elm_vinculo.text
                                    # print('Elemento retornado:',nome_vinculo)
                                    retry(ActionChains(browser).click(elm_vinculo).perform(),
                                        wait_ms=100,
                                        limit=limite,
                                        on_exhaust=(f'Problema ao clicar no link do nome. {limite} tentativas sem sucesso.'))            
                                except Exception as e:
                                    print('Erro ao achar o link do nome com múltiplos resultados')
                                    return np.NaN, NOME, np.NaN, e, browser
                                force_break_loop = True
                                break
                        except Exception as e2:
                            traceback_str = ''.join(traceback.format_tb(e2.__traceback__))
                            print('Erro ao procurar vínculo com currículos achados')    
                            print(e2,traceback_str)
                        ## Caso percorra todos elementos da lista e não encontre vínculo adiciona à dúvidas quanto ao nome
                        if m==(qte_resultados):
                            print(f'Não encontrada nenhuma referência à {instituicao} ou ao {unidade} ou ao termo {termo}')
                            duvidas.append(NOME)
                            # clear_output(wait=True)
                            # browser.quit()
                            continue
                    if force_break_loop:
                        break
                try:
                    prox = browser.find_element(By.PARTIAL_LINK_TEXT, 'próximo')
                    prox.click()
                except:
                    continue
        try:
            elm_vinculo.text
            # print(f'Nomes: {NOME} | {elm_vinculo.text}')
        except:
            return np.NaN, NOME, np.NaN, 'Vínculo não encontrado', browser
    except Exception as err:
        print('Erro ao sair da função procurar_vinculos()')
        print('Conteúdo do erro:',err)
        return np.NaN, NOME, np.NaN, err, browser
    return elm_vinculo, np.NaN, np.NaN, np.NaN, browser

def listar_idlattes(df_secoes):
    df_idlattes = df_secoes[df_secoes['ROTULOS']=='ID Lattes:']
    return df_idlattes

def listar_artigos(df_secoes):
    df_idlattes = df_secoes[df_secoes['ROTULOS']=='Artigos completos publicados em periódicos']
    return df_idlattes

def contar_artigos(df_secoes):
    def count_year_occurrences(content):
        """Count the number of occurrences of four-digit years followed by a period in the given string.""" 
        if not isinstance(content, (str, bytes)):
            content = ' '.join(map(str, content))
        pattern = r'\b\d{4}\.'
        return len(re.findall(pattern, content))

    def extract_citations(content):
        if not isinstance(content, (str, bytes)):
            content = ' '.join(map(str, content))
        citation_pattern = r'Citações:([\d\|]+),'
        all_citations = re.findall(citation_pattern, content)
        total_citations = 0
        for citation_group in all_citations:
            numbers = map(int, citation_group.split('|'))
            total_citations += sum(numbers)
        return len(all_citations), total_citations

    df_secoes_contadas = df_secoes.copy()
    df_secoes_contadas['QTE_ARTIGOS'] = df_secoes['CONTEUDOS'].apply(count_year_occurrences)
    df_secoes_contadas['ARTIGOS_CITADOS'], df_secoes['SOMA_CITACOES'] = zip(*df_secoes['CONTEUDOS'].apply(extract_citations))

    return df_secoes_contadas

# Extração com BeaultifulSoap
def parse_header(soup):
    title = soup.title.string if soup.title else "Unknown"
    meta_keywords = soup.find("meta", {"http-equiv": "keywords"})["content"] if soup.find("meta", {"http-equiv": "keywords"}) else "Unknown"
    return {"title": title, "meta_keywords": meta_keywords}

def parse_h1_elements(soup, parent_node, graph):
    h1_elements = soup.find_all('h1', {'tabindex': '0'})
    for elem in h1_elements:
        h1_text = elem.text.strip()
        h1_node = Node("H1Element", text=h1_text)
        graph.create(h1_node)
        relation = Relationship(parent_node, "HAS_SECTION", h1_node)
        graph.create(relation)

def persist_to_neo4j(header_data):
    graph = Graph("bolt://localhost:7687", auth=("neo4j", "password"))  # Substitua pelo seu endpoint e credenciais
    header_node = Node("Curriculum", title=header_data['title'], meta_keywords=header_data['meta_keywords'])
    graph.create(header_node)
    return header_node

# if __name__ == "__main__":
#     # Exemplo de entrada HTML
#     html_content = '''
#     <html lang="pt-BR">
#         <head>
#             <title>Currículo do Sistema de Currículos Lattes (Raimir Holanda Filho)</title>
#             <meta content="currículo,curriculo,curriculum,cv,vitae,lattes,produção,producao,científica,cientifica,Brasil" http-equiv="keywords">
#         </head>
#         <body>
#             <h1 tabindex="0">Educação</h1>
#             <h1 tabindex="0">Publicações</h1>
#             <!-- ... -->
#         </body>
#     </html>
#     '''
#     # Inicialização do BeautifulSoup
#     soup = BeautifulSoup(html_content, 'html.parser')

#     # Extração e Persistência do cabeçalho
#     header_data = parse_header(soup)
#     header_node = persist_to_neo4j(header_data)
    
#     # Extração e Persistência dos elementos H1
#     graph = Graph("bolt://localhost:7687", auth=("neo4j", "password"))
#     parse_h1_elements(soup, header_node, graph)

In [ ]:
drives=['C:/Users/','E:/','./home/']
pastas=['marcos.aires/', 'marco/']
pastasraiz=['kgfioce','fioce']
caminho=try_folders(drives,pastas,pastasraiz)

preparar_pastas(caminho)
try_amb()
try_gpu()
try_browser(caminho)

In [ ]:
def conectar_busca():    
    print(f'Conectando com o servidor do CNPq...')
    # print(f'Iniciada extração de {len(lista_nomes)} currículos')
    
    ## https://www.selenium.dev/documentation/pt-br/webdriver/browser_manipulation/
    options   = Options()
    # options.add_argument("--headless")
    browser   = webdriver.Chrome(options=options)
    url_buscaespecialista = 'http://buscatextual.cnpq.br/buscatextual/busca.do?buscarDoutores=true&buscarDemais=true&textoBusca='
    browser.get(url_buscaespecialista) # acessa a url de busca do CNPQ   
    
    browser.set_window_position(-20, -10)
    browser.set_window_size(170, 1896)
    browser.mouse = webdriver.ActionChains(browser)
    # url        = browser.command_executor._url #"http://127.0.0.1:60622/hub"
    # session_id = browser.session_id            #'4e167f26-dc1d-4f51-a207-f761eaf73c31'

    # return browser, url, session_id
    return browser

In [ ]:
from bs4 import BeautifulSoup
from py2neo import Graph, Node

def parse_parsoninfo(soup):
    # Localizar o elemento de link que contém o título do currículo
    link_element = soup.find("a", {"href": lambda x: x and "abreDetalhe" in x})

    # Extrair o texto do link para usar como título do nó
    node_title = link_element.text if link_element else "Unknown"

    # Localizar o elemento div contendo as propriedades
    properties_div = soup.find("div", {"class": "resultado"})

    # Localizar o elemento li que contém as informações
    li_element = soup.find("li")

    # Inicializar um dicionário para armazenar as propriedades
    properties = {}

    # Extrair e armazenar as propriedades relevantes
    if properties_div:
        properties['Nacionalidade'] = 'Brasil'
        properties['Cargo'] = properties_div.find("span", {"class": "tooltip coautor"}).text if properties_div.find("span", {"class": "tooltip coautor"}) else 'Desconhecido'
        properties['Titulação'] = properties_div.contents[-4] if len(properties_div.contents) > 4 else 'Desconhecido'

        # Extração de nome e identificador único
        a_element = li_element.find("a")
        properties["Nome"] = a_element.text

        # Extração de Nacionalidade
        img_element = li_element.find("img")
        properties["Nacionalidade"] = img_element.get("title") or img_element.get("alt")

        # Extração de Bolsa de Produtividade
        span_element = li_element.find("span", {"class": "tooltip coautor"})
        properties["Bolsista_Prod_Pesquisa"] = span_element.get("alt")

        # Extração de Nacionalidade
        img_element = li_element.find("img")
        properties["Nacionalidade"] = img_element.get("title") or img_element.get("alt")

        # Extração de Bolsa de Produtividade
        span_element = li_element.find("span", {"class": "tooltip coautor"})
        properties["Bolsista_Prod_Pesquisa"] = span_element.get("alt")

        # Extração de Títulos Acadêmicos e outras informações



    return node_title, properties

In [ ]:
def extrair_lista(lista, mestres=True, assunto=False):
    sucesso=[]
    falhas=[]
    duvidas=[]
    tipo_erro=[]
    curriculos=[]
    rotulos=[]
    conteudos=[]

    delay=10
    limite=3
    instituicao = 'Fundação Oswaldo Cruz'
    unidade     = 'Fiocruz Ceará'
    termo       = 'Ministerio da Saude'

    t0 = time.time()
    browser = conectar_busca()
    for NOME in lista:
        try:
            preencher_busca(browser, delay, NOME)
            elemento_achado, nome_falha, duvida, erro, browser = procurar_vinculos(NOME, instituicao, unidade, termo, browser, delay, limite)
            link_nome     = achar_busca(browser, delay)
            window_before = browser.current_window_handle
            
            if str(elemento_achado) == 'nan':
                print('Vínculo não encontrado, passando ao próximo nome...')
                falhas.append(nome_falha)
                duvidas.append(duvida)
                tipo_erro.append(erro)
                # print(nome_falha)
                # print(erro)
                # clear_output(wait=True)
                raise Exception
            print('Vínculo encontrado no currículo de nome:',elemento_achado.text)

            ## Clicar no botão abrir currículo e mudar de aba
            try:
                ## Aguarda, encontra, clica em buscar nome
                link_nome    = achar_busca(browser, delay)
                nome_buscado = []
                nome_achado  = []
                nome_buscado.append(NOME)
                
                if link_nome.text == None:
                    xpath_nome = '/html/body/form/div/div[4]/div/div/div/div[3]/div/div[3]/ol/li'
                    # 'Stale file handle'
                    print('Ainda sem resposta do servidor, tentando novamente...')
                    retry(WebDriverWait(browser, delay).until(
                        EC.element_to_be_clickable((By.XPATH, xpath_nome))),
                    #    expected_ex_type=ZeroDivisionError, 
                    wait_ms=200,
                    limit=limite, 
                    #    logger=logger, 
                    on_exhaust=(f'Problema ao acessar ao servidor do CNpQ função definir_filtros(). {limite} tentativas sem sucesso.'))
                try:
                    ActionChains(browser).click(link_nome).perform()
                    nome_achado.append(link_nome.text)
                except:
                    print(f'Currículo não encontrado para: {NOME}.')
                    return
                
                retry(WebDriverWait(browser, delay).until(
                    EC.element_to_be_clickable((By.CSS_SELECTOR, "#idbtnabrircurriculo"))),
                    #    expected_ex_type=ZeroDivisionError, 
                    wait_ms=200,
                    limit=limite, 
                    #    logger=logger, 
                    on_exhaust=(f'Problema ao acessar ao servidor do CNpQ função definir_filtros(). {limite} tentativas sem sucesso.'))   
                
                # Clicar botão para abrir o currículo
                btn_abrir_curriculo = WebDriverWait(browser, delay).until(
                    EC.element_to_be_clickable((By.CSS_SELECTOR, "#idbtnabrircurriculo")))
                time.sleep(0.2)
                ActionChains(browser).click(btn_abrir_curriculo).perform()

                ## Gerenciamento das janelas abertas no browser
                WebDriverWait(browser, delay).until(EC.number_of_windows_to_be(2))
                window_after = browser.window_handles
                new_window   = [x for x in window_after if x != window_before][0]
                browser.switch_to.window(new_window)

                # Pega o código fonte da página
                page_source = browser.page_source

                # Usa BeautifulSoup para analisar
                soup = BeautifulSoup(page_source, 'html.parser')

                # Extração e Persistência do cabeçalho
                header_data = parse_header(soup)
                header_node = persist_to_neo4j(header_data)
                
                # Extração e Persistência dos elementos H1
                graph = Graph("bolt://localhost:7687", auth=("neo4j", "password"))
                parse_h1_elements(soup, header_node, graph)
                graph, cv_node, properties = parse_parsoninfo(soup)                

            except Exception as e:
                print('Erro',e)
                print('Tentando nova requisição ao servidor')
                time.sleep(1)
                btn_abrir_curriculo = WebDriverWait(browser, delay).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, "#idbtnabrircurriculo")))
                ActionChains(browser).click(btn_abrir_curriculo).perform()
                WebDriverWait(browser, delay).until(EC.number_of_windows_to_be(2))

                ## Gerenciamento das janelas abertas no browser
                window_after = browser.window_handles
                new_window   = [x for x in window_after if x != window_before][0]
                browser.switch_to.window(new_window)
                time.sleep(1)

            sucesso.append(NOME)

        except:
            print(f'Currículo não encontrado: {NOME}')
            browser.back()
            continue

    df_dados =pd.DataFrame({
        'CURRICULO': pd.Series(curriculos),
        'ROTULOS': pd.Series(rotulos),
        'CONTEUDOS': pd.Series(conteudos),
            })
    
    t6=time.time()
    print('='*95)
    # print(f' {len(sucesso)} currículos extraídos com sucesso')
    print(f' Tempo total para extrair {len(conteudos)} seções dos currículos: {tempo(t0,t6)}')
    # print('='*95)
    browser.quit()
    
    return df_dados, sucesso  

# Iniciar execução

In [ ]:
lista = ['Raimir Holanda Filho', 'Carlos Jose Araujo Pinheiro','Antonio Marcos Aires Barbosa']
df_dados, sucesso = extrair_lista(lista, mestres=True, assunto=False)

In [ ]:
browser.quit()